In [1]:
## This file implements neural networks and logistic regression on p0040presabsSTCC_quant.
## For fully-connected neural networks, the accuracy is 75.32%.
## We further implement the over-sampling method and the combination of over- and under-sampling method, in case of imbalanced data.
## For fully-connected neural networks, the accuracy is 88.14% for combination data
## and 79.8% for over-sampling data.
## To improve neural network performance, we implement lasso regression to select important features.
## With the new dataset, the accuracy becomes 79.22% without balanced methods, 74.58% for combination, and 77.78% for over-sampling data.
## For logistic regression, the accuracies are 76.62% without balanced methods, 94.92 for combination data and 85.86% for over-sampling data.
## For random forest, the accuracy is 76.62% without balanced methods, 100% for combination, and 86.87% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 82.57% without balanced methods, 91.93% for combination data, and 83.09% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0040presabsSTCC_quant.csv')
df.shape

(255, 375)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.814000
1      0.365125
2      0.803875
3      1.088125
4      1.081750
5      1.112875
6      0.817875
7      0.398250
8      0.569500
9      0.526125
10     0.351625
11     0.814875
12     0.590875
13     0.373875
14     0.922625
15     0.955750
16     0.383500
17     0.379625
18     0.689625
19     0.186750
20     0.351750
21     0.364250
22     0.306125
23     1.112250
24     0.963000
25     0.159500
26     0.824750
27     0.196000
28     0.590875
29     0.830375
         ...   
225    0.715500
226    0.469000
227    0.670400
228    0.746000
229    0.969000
230    0.577125
231    0.627375
232    0.930500
233    0.478875
234    0.602500
235    0.900125
236    0.646875
237    0.847125
238    0.461500
239    0.258375
240    0.515875
241    0.466875
242    0.275500
243    0.542125
244    0.486625
245    0.765375
246    0.845625
247    0.530875
248    0.894625
249    0.457250
250    0.868000
251    0.550500
252    0.685125
253    0.939500
254    0.926500
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,...,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno,ST,CC
0,107,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0.814000,5,5
1,109,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0.365125,8,8
2,115,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.803875,5,5
3,120335,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1.088125,5,5
4,120337,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1.081750,5,5


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

1    165
0     90
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 375)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 374)

In [13]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno,ST,CC
0,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,5,5
1,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,8,8
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
3,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
4,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 373) (255,)


In [15]:
# split into train, test data (original)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

Using TensorFlow backend.


In [18]:
#### neural network
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [19]:
model1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/100
178/178 [==============================] - 0s 1ms/step - loss: 13.7483 - accuracy: 0.5169 - val_loss: 0.6851 - val_accuracy: 0.6104
Epoch 2/100
178/178 [==============================] - 0s 102us/step - loss: 0.6833 - accuracy: 0.6124 - val_loss: 0.6802 - val_accuracy: 0.6234
Epoch 3/100
178/178 [==============================] - 0s 59us/step - loss: 0.6613 - accuracy: 0.6573 - val_loss: 0.6683 - val_accuracy: 0.6104
Epoch 4/100
178/178 [==============================] - 0s 73us/step - loss: 0.6431 - accuracy: 0.6629 - val_loss: 0.6830 - val_accuracy: 0.5974
Epoch 5/100
178/178 [==============================] - 0s 175us/step - loss: 0.6314 - accuracy: 0.6798 - val_loss: 0.6752 - val_accuracy: 0.5844
Epoch 6/100
178/178 [==============================] - 0s 100us/step - loss: 0.6083 - accuracy: 0.6910 - val_loss: 0.6569 - val_accuracy: 0.6104
Epoch 7/100
178/178 [==============================] - 0s 105us/step - loss: 0.6084 - ac

Epoch 57/100
178/178 [==============================] - 0s 113us/step - loss: 0.4858 - accuracy: 0.8202 - val_loss: 0.6844 - val_accuracy: 0.7532
Epoch 58/100
178/178 [==============================] - 0s 105us/step - loss: 0.5038 - accuracy: 0.7921 - val_loss: 0.6708 - val_accuracy: 0.7662
Epoch 59/100
178/178 [==============================] - 0s 237us/step - loss: 0.4849 - accuracy: 0.8090 - val_loss: 0.6883 - val_accuracy: 0.7403
Epoch 60/100
178/178 [==============================] - 0s 177us/step - loss: 0.4880 - accuracy: 0.8090 - val_loss: 0.6910 - val_accuracy: 0.7403
Epoch 61/100
178/178 [==============================] - 0s 175us/step - loss: 0.4866 - accuracy: 0.8034 - val_loss: 0.7198 - val_accuracy: 0.7013
Epoch 62/100
178/178 [==============================] - 0s 262us/step - loss: 0.4907 - accuracy: 0.7978 - val_loss: 0.6784 - val_accuracy: 0.7662
Epoch 63/100
178/178 [==============================] - 0s 331us/step - loss: 0.4755 - accuracy: 0.8146 - val_loss: 0.6878 -

In [21]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

77/77 [==============================] - 0s 247us/step
test accuracy: 75.32%


In [111]:
#### improve neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [112]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [113]:
model2.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 758us/step - loss: 10.3547 - accuracy: 0.6067 - val_loss: 1.0161 - val_accuracy: 0.6494
Epoch 2/1000
178/178 [==============================] - 0s 154us/step - loss: 0.9258 - accuracy: 0.6742 - val_loss: 1.0483 - val_accuracy: 0.6883
Epoch 3/1000
178/178 [==============================] - 0s 163us/step - loss: 0.7144 - accuracy: 0.7135 - val_loss: 0.6280 - val_accuracy: 0.6623
Epoch 4/1000
178/178 [==============================] - 0s 171us/step - loss: 0.6667 - accuracy: 0.7247 - val_loss: 0.6147 - val_accuracy: 0.7143
Epoch 5/1000
178/178 [==============================] - 0s 160us/step - loss: 0.5787 - accuracy: 0.7191 - val_loss: 0.6152 - val_accuracy: 0.7143
Epoch 6/1000
178/178 [==============================] - 0s 184us/step - loss: 0.5753 - accuracy: 0.7416 - val_loss: 0.9429 - val_accuracy: 0.7273
Epoch 7/1000
178/178 [==============================] - 0s 183us/step - loss: 

178/178 [==============================] - 0s 162us/step - loss: 0.4814 - accuracy: 0.8034 - val_loss: 0.6331 - val_accuracy: 0.7532
Epoch 57/1000
178/178 [==============================] - 0s 184us/step - loss: 0.4615 - accuracy: 0.8202 - val_loss: 0.7839 - val_accuracy: 0.7013
Epoch 58/1000
178/178 [==============================] - 0s 162us/step - loss: 0.5247 - accuracy: 0.8202 - val_loss: 0.6289 - val_accuracy: 0.7532
Epoch 59/1000
178/178 [==============================] - 0s 178us/step - loss: 0.4532 - accuracy: 0.8202 - val_loss: 0.6283 - val_accuracy: 0.7662
Epoch 60/1000
178/178 [==============================] - 0s 162us/step - loss: 0.4270 - accuracy: 0.8202 - val_loss: 0.6353 - val_accuracy: 0.7792
Epoch 61/1000
178/178 [==============================] - 0s 155us/step - loss: 0.4164 - accuracy: 0.8202 - val_loss: 0.6239 - val_accuracy: 0.7662
Epoch 62/1000
178/178 [==============================] - 0s 155us/step - loss: 0.4453 - accuracy: 0.8146 - val_loss: 0.6411 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 154us/step - loss: 0.3983 - accuracy: 0.8258 - val_loss: 0.6613 - val_accuracy: 0.7273
Epoch 113/1000
178/178 [==============================] - 0s 148us/step - loss: 0.3981 - accuracy: 0.8258 - val_loss: 0.6663 - val_accuracy: 0.7273
Epoch 114/1000
178/178 [==============================] - 0s 179us/step - loss: 0.3979 - accuracy: 0.8258 - val_loss: 0.6633 - val_accuracy: 0.7403
Epoch 115/1000
178/178 [==============================] - 0s 186us/step - loss: 0.3964 - accuracy: 0.8258 - val_loss: 0.6613 - val_accuracy: 0.7532
Epoch 116/1000
178/178 [==============================] - 0s 152us/step - loss: 0.3964 - accuracy: 0.8258 - val_loss: 0.6562 - val_accuracy: 0.7403
Epoch 117/1000
178/178 [==============================] - 0s 162us/step - loss: 0.3964 - accuracy: 0.8258 - val_loss: 0.6671 - val_accuracy: 0.7403
Epoch 118/1000
178/178 [==============================] - 0s 145us/step - loss: 0.3946 - accuracy: 0.8258 - val_

178/178 [==============================] - 0s 154us/step - loss: 0.3670 - accuracy: 0.8371 - val_loss: 0.6829 - val_accuracy: 0.7403
Epoch 168/1000
178/178 [==============================] - 0s 153us/step - loss: 0.3686 - accuracy: 0.8371 - val_loss: 0.6772 - val_accuracy: 0.7532
Epoch 169/1000
178/178 [==============================] - 0s 164us/step - loss: 0.3654 - accuracy: 0.8315 - val_loss: 0.7153 - val_accuracy: 0.7532
Epoch 170/1000
178/178 [==============================] - 0s 174us/step - loss: 0.3688 - accuracy: 0.8315 - val_loss: 0.6884 - val_accuracy: 0.7403
Epoch 171/1000
178/178 [==============================] - 0s 163us/step - loss: 0.3675 - accuracy: 0.8315 - val_loss: 0.6825 - val_accuracy: 0.7532
Epoch 172/1000
178/178 [==============================] - 0s 155us/step - loss: 0.3645 - accuracy: 0.8315 - val_loss: 0.6865 - val_accuracy: 0.7532
Epoch 173/1000
178/178 [==============================] - 0s 139us/step - loss: 0.3632 - accuracy: 0.8315 - val_loss: 0.7126 - 

178/178 [==============================] - 0s 143us/step - loss: 0.3419 - accuracy: 0.8539 - val_loss: 0.7216 - val_accuracy: 0.7792
Epoch 223/1000
178/178 [==============================] - 0s 145us/step - loss: 0.3434 - accuracy: 0.8539 - val_loss: 0.7275 - val_accuracy: 0.7792
Epoch 224/1000
178/178 [==============================] - 0s 149us/step - loss: 0.3510 - accuracy: 0.8539 - val_loss: 0.7245 - val_accuracy: 0.7662
Epoch 225/1000
178/178 [==============================] - 0s 151us/step - loss: 0.3451 - accuracy: 0.8539 - val_loss: 0.7719 - val_accuracy: 0.7403
Epoch 226/1000
178/178 [==============================] - 0s 148us/step - loss: 0.3440 - accuracy: 0.8596 - val_loss: 0.7513 - val_accuracy: 0.7532
Epoch 227/1000
178/178 [==============================] - 0s 145us/step - loss: 0.3435 - accuracy: 0.8539 - val_loss: 0.7375 - val_accuracy: 0.7532
Epoch 228/1000
178/178 [==============================] - 0s 144us/step - loss: 0.3428 - accuracy: 0.8539 - val_loss: 0.7432 - 

...

178/178 [==============================] - 0s 145us/step - loss: 0.2793 - accuracy: 0.8764 - val_loss: 1.3557 - val_accuracy: 0.7273
Epoch 773/1000
178/178 [==============================] - 0s 158us/step - loss: 0.2792 - accuracy: 0.8764 - val_loss: 1.3372 - val_accuracy: 0.7143
Epoch 774/1000
178/178 [==============================] - 0s 155us/step - loss: 0.2797 - accuracy: 0.8764 - val_loss: 1.3724 - val_accuracy: 0.7143
Epoch 775/1000
178/178 [==============================] - 0s 150us/step - loss: 0.2801 - accuracy: 0.8764 - val_loss: 1.3863 - val_accuracy: 0.7273
Epoch 776/1000
178/178 [==============================] - 0s 158us/step - loss: 0.2823 - accuracy: 0.8764 - val_loss: 1.3832 - val_accuracy: 0.7143
Epoch 777/1000
178/178 [==============================] - 0s 161us/step - loss: 0.2792 - accuracy: 0.8764 - val_loss: 1.3813 - val_accuracy: 0.7143
Epoch 778/1000
178/178 [==============================] - 0s 158us/step - loss: 0.2783 - accuracy: 0.8764 - val_loss: 1.3724 - 

178/178 [==============================] - 0s 173us/step - loss: 0.2859 - accuracy: 0.8708 - val_loss: 1.4122 - val_accuracy: 0.7273
Epoch 828/1000
178/178 [==============================] - 0s 181us/step - loss: 0.2859 - accuracy: 0.8708 - val_loss: 1.4191 - val_accuracy: 0.7273
Epoch 829/1000
178/178 [==============================] - 0s 218us/step - loss: 0.2859 - accuracy: 0.8708 - val_loss: 1.4137 - val_accuracy: 0.7273
Epoch 830/1000
178/178 [==============================] - 0s 184us/step - loss: 0.2857 - accuracy: 0.8708 - val_loss: 1.4170 - val_accuracy: 0.7273
Epoch 831/1000
178/178 [==============================] - 0s 198us/step - loss: 0.2857 - accuracy: 0.8708 - val_loss: 1.4128 - val_accuracy: 0.7273
Epoch 832/1000
178/178 [==============================] - 0s 231us/step - loss: 0.2857 - accuracy: 0.8708 - val_loss: 1.4172 - val_accuracy: 0.7273
Epoch 833/1000
178/178 [==============================] - 0s 200us/step - loss: 0.2855 - accuracy: 0.8708 - val_loss: 1.4185 - 

178/178 [==============================] - 0s 145us/step - loss: 0.2756 - accuracy: 0.8764 - val_loss: 1.4522 - val_accuracy: 0.7273
Epoch 883/1000
178/178 [==============================] - 0s 157us/step - loss: 0.2753 - accuracy: 0.8764 - val_loss: 1.4436 - val_accuracy: 0.7273
Epoch 884/1000
178/178 [==============================] - 0s 153us/step - loss: 0.2751 - accuracy: 0.8820 - val_loss: 1.4363 - val_accuracy: 0.7273
Epoch 885/1000
178/178 [==============================] - 0s 151us/step - loss: 0.2762 - accuracy: 0.8764 - val_loss: 1.4426 - val_accuracy: 0.7273
Epoch 886/1000
178/178 [==============================] - 0s 144us/step - loss: 0.2765 - accuracy: 0.8820 - val_loss: 1.4422 - val_accuracy: 0.7273
Epoch 887/1000
178/178 [==============================] - 0s 142us/step - loss: 0.2759 - accuracy: 0.8764 - val_loss: 1.4510 - val_accuracy: 0.7273
Epoch 888/1000
178/178 [==============================] - 0s 145us/step - loss: 0.2756 - accuracy: 0.8764 - val_loss: 1.4438 - 

178/178 [==============================] - 0s 147us/step - loss: 0.2769 - accuracy: 0.8764 - val_loss: 1.4810 - val_accuracy: 0.7273
Epoch 938/1000
178/178 [==============================] - 0s 147us/step - loss: 0.2746 - accuracy: 0.8764 - val_loss: 1.4767 - val_accuracy: 0.7273
Epoch 939/1000
178/178 [==============================] - 0s 151us/step - loss: 0.2752 - accuracy: 0.8820 - val_loss: 1.4868 - val_accuracy: 0.7143
Epoch 940/1000
178/178 [==============================] - 0s 150us/step - loss: 0.2761 - accuracy: 0.8764 - val_loss: 1.4890 - val_accuracy: 0.7143
Epoch 941/1000
178/178 [==============================] - 0s 146us/step - loss: 0.2762 - accuracy: 0.8708 - val_loss: 1.4938 - val_accuracy: 0.7143
Epoch 942/1000
178/178 [==============================] - 0s 148us/step - loss: 0.2825 - accuracy: 0.8764 - val_loss: 1.4877 - val_accuracy: 0.7143
Epoch 943/1000
178/178 [==============================] - 0s 145us/step - loss: 0.2772 - accuracy: 0.8764 - val_loss: 1.4864 - 

178/178 [==============================] - 0s 158us/step - loss: 0.2841 - accuracy: 0.8708 - val_loss: 1.5216 - val_accuracy: 0.7273
Epoch 993/1000
178/178 [==============================] - 0s 157us/step - loss: 0.2840 - accuracy: 0.8708 - val_loss: 1.5169 - val_accuracy: 0.7273
Epoch 994/1000
178/178 [==============================] - 0s 159us/step - loss: 0.2837 - accuracy: 0.8708 - val_loss: 1.5205 - val_accuracy: 0.7273
Epoch 995/1000
178/178 [==============================] - 0s 153us/step - loss: 0.2840 - accuracy: 0.8708 - val_loss: 1.5146 - val_accuracy: 0.7273
Epoch 996/1000
178/178 [==============================] - 0s 162us/step - loss: 0.2842 - accuracy: 0.8708 - val_loss: 1.5266 - val_accuracy: 0.7273
Epoch 997/1000
178/178 [==============================] - 0s 152us/step - loss: 0.2842 - accuracy: 0.8708 - val_loss: 1.5273 - val_accuracy: 0.7273
Epoch 998/1000
178/178 [==============================] - 0s 146us/step - loss: 0.2840 - accuracy: 0.8708 - val_loss: 1.5181 - 

In [114]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 100us/step
test accuracy: 72.73%


In [26]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 105), (1, 91)]


In [27]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 165), (1, 165)]


In [28]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [29]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [30]:
############# Fully-Connected Neural Network ################

In [31]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [32]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [33]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [34]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 137 samples, validate on 59 samples
Epoch 1/1000
137/137 [==============================] - 0s 1ms/step - loss: 3.5949 - accuracy: 0.6715 - val_loss: 1.5293 - val_accuracy: 0.7966
Epoch 2/1000
137/137 [==============================] - 0s 374us/step - loss: 1.3001 - accuracy: 0.7956 - val_loss: 0.7531 - val_accuracy: 0.6271
Epoch 3/1000
137/137 [==============================] - 0s 204us/step - loss: 0.6812 - accuracy: 0.7299 - val_loss: 1.6419 - val_accuracy: 0.7966
Epoch 4/1000
137/137 [==============================] - 0s 236us/step - loss: 1.2408 - accuracy: 0.8248 - val_loss: 0.5486 - val_accuracy: 0.7458
Epoch 5/1000
137/137 [==============================] - 0s 334us/step - loss: 0.5008 - accuracy: 0.7956 - val_loss: 0.5222 - val_accuracy: 0.7627
Epoch 6/1000
137/137 [==============================] - 0s 333us/step - loss: 0.5024 - accuracy: 0.7810 - val_loss: 0.5168 - val_accuracy: 0.7797
Epoch 7/1000
137/137 [==============================] - 0s 407us/step - loss: 0.4

137/137 [==============================] - 0s 238us/step - loss: 0.2957 - accuracy: 0.8540 - val_loss: 0.3643 - val_accuracy: 0.8136
Epoch 57/1000
137/137 [==============================] - 0s 241us/step - loss: 0.3124 - accuracy: 0.7810 - val_loss: 0.3632 - val_accuracy: 0.8136
Epoch 58/1000
137/137 [==============================] - 0s 169us/step - loss: 0.3087 - accuracy: 0.7956 - val_loss: 0.3652 - val_accuracy: 0.8305
Epoch 59/1000
137/137 [==============================] - 0s 166us/step - loss: 0.3063 - accuracy: 0.8248 - val_loss: 0.3628 - val_accuracy: 0.7966
Epoch 60/1000
137/137 [==============================] - 0s 191us/step - loss: 0.3005 - accuracy: 0.8394 - val_loss: 0.3693 - val_accuracy: 0.7966
Epoch 61/1000
137/137 [==============================] - 0s 168us/step - loss: 0.2936 - accuracy: 0.8394 - val_loss: 0.3651 - val_accuracy: 0.7966
Epoch 62/1000
137/137 [==============================] - 0s 166us/step - loss: 0.2897 - accuracy: 0.8321 - val_loss: 0.3649 - val_ac

Epoch 112/1000
137/137 [==============================] - 0s 259us/step - loss: 0.2910 - accuracy: 0.8540 - val_loss: 0.3597 - val_accuracy: 0.7797
Epoch 113/1000
137/137 [==============================] - 0s 291us/step - loss: 0.2786 - accuracy: 0.8467 - val_loss: 0.3384 - val_accuracy: 0.8136
Epoch 114/1000
137/137 [==============================] - 0s 206us/step - loss: 0.2718 - accuracy: 0.8613 - val_loss: 0.3411 - val_accuracy: 0.8136
Epoch 115/1000
137/137 [==============================] - 0s 259us/step - loss: 0.2654 - accuracy: 0.8540 - val_loss: 0.3526 - val_accuracy: 0.8305
Epoch 116/1000
137/137 [==============================] - 0s 267us/step - loss: 0.2822 - accuracy: 0.8686 - val_loss: 0.3432 - val_accuracy: 0.8136
Epoch 117/1000
137/137 [==============================] - 0s 183us/step - loss: 0.2798 - accuracy: 0.8467 - val_loss: 0.4631 - val_accuracy: 0.7119
Epoch 118/1000
137/137 [==============================] - 0s 108us/step - loss: 0.2829 - accuracy: 0.8394 - val_

Epoch 167/1000
137/137 [==============================] - 0s 201us/step - loss: 0.2685 - accuracy: 0.8540 - val_loss: 0.3204 - val_accuracy: 0.8136
Epoch 168/1000
137/137 [==============================] - 0s 180us/step - loss: 0.2663 - accuracy: 0.8613 - val_loss: 0.3211 - val_accuracy: 0.8136
Epoch 169/1000
137/137 [==============================] - 0s 291us/step - loss: 0.2661 - accuracy: 0.8467 - val_loss: 0.3199 - val_accuracy: 0.8136
Epoch 170/1000
137/137 [==============================] - 0s 152us/step - loss: 0.2664 - accuracy: 0.8540 - val_loss: 0.3161 - val_accuracy: 0.8136
Epoch 171/1000
137/137 [==============================] - 0s 116us/step - loss: 0.2600 - accuracy: 0.8540 - val_loss: 0.3270 - val_accuracy: 0.8136
Epoch 172/1000
137/137 [==============================] - 0s 116us/step - loss: 0.2562 - accuracy: 0.8540 - val_loss: 0.3140 - val_accuracy: 0.8136
Epoch 173/1000
137/137 [==============================] - 0s 105us/step - loss: 0.2580 - accuracy: 0.8540 - val_

137/137 [==============================] - 0s 242us/step - loss: 0.2526 - accuracy: 0.8540 - val_loss: 0.2900 - val_accuracy: 0.8136
Epoch 223/1000
137/137 [==============================] - 0s 166us/step - loss: 0.2511 - accuracy: 0.8540 - val_loss: 0.2898 - val_accuracy: 0.8136
Epoch 224/1000
137/137 [==============================] - 0s 232us/step - loss: 0.2569 - accuracy: 0.8613 - val_loss: 0.2909 - val_accuracy: 0.8136
Epoch 225/1000
137/137 [==============================] - 0s 223us/step - loss: 0.2574 - accuracy: 0.8540 - val_loss: 0.2970 - val_accuracy: 0.8136
Epoch 226/1000
137/137 [==============================] - 0s 216us/step - loss: 0.2573 - accuracy: 0.8540 - val_loss: 0.2907 - val_accuracy: 0.8136
Epoch 227/1000
137/137 [==============================] - 0s 236us/step - loss: 0.2561 - accuracy: 0.8613 - val_loss: 0.2908 - val_accuracy: 0.8136
Epoch 228/1000
137/137 [==============================] - 0s 334us/step - loss: 0.2578 - accuracy: 0.8540 - val_loss: 0.2974 - 

...

137/137 [==============================] - 0s 107us/step - loss: 0.2082 - accuracy: 0.8832 - val_loss: 0.2264 - val_accuracy: 0.8644
Epoch 773/1000
137/137 [==============================] - 0s 108us/step - loss: 0.2078 - accuracy: 0.8832 - val_loss: 0.2260 - val_accuracy: 0.8814
Epoch 774/1000
137/137 [==============================] - 0s 111us/step - loss: 0.2078 - accuracy: 0.8832 - val_loss: 0.2251 - val_accuracy: 0.8814
Epoch 775/1000
137/137 [==============================] - 0s 115us/step - loss: 0.2080 - accuracy: 0.8832 - val_loss: 0.2258 - val_accuracy: 0.8814
Epoch 776/1000
137/137 [==============================] - 0s 112us/step - loss: 0.2081 - accuracy: 0.8832 - val_loss: 0.2252 - val_accuracy: 0.8814
Epoch 777/1000
137/137 [==============================] - 0s 109us/step - loss: 0.2081 - accuracy: 0.8832 - val_loss: 0.2260 - val_accuracy: 0.8814
Epoch 778/1000
137/137 [==============================] - 0s 107us/step - loss: 0.2078 - accuracy: 0.8832 - val_loss: 0.2273 - 

137/137 [==============================] - 0s 99us/step - loss: 0.2068 - accuracy: 0.8832 - val_loss: 0.2281 - val_accuracy: 0.8644
Epoch 828/1000
137/137 [==============================] - 0s 113us/step - loss: 0.2070 - accuracy: 0.8832 - val_loss: 0.2281 - val_accuracy: 0.8644
Epoch 829/1000
137/137 [==============================] - 0s 139us/step - loss: 0.2067 - accuracy: 0.8832 - val_loss: 0.2290 - val_accuracy: 0.8644
Epoch 830/1000
137/137 [==============================] - 0s 113us/step - loss: 0.2067 - accuracy: 0.8832 - val_loss: 0.2278 - val_accuracy: 0.8644
Epoch 831/1000
137/137 [==============================] - 0s 124us/step - loss: 0.2068 - accuracy: 0.8832 - val_loss: 0.2277 - val_accuracy: 0.8644
Epoch 832/1000
137/137 [==============================] - 0s 102us/step - loss: 0.2066 - accuracy: 0.8832 - val_loss: 0.2272 - val_accuracy: 0.8644
Epoch 833/1000
137/137 [==============================] - 0s 108us/step - loss: 0.2065 - accuracy: 0.8832 - val_loss: 0.2275 - v

137/137 [==============================] - 0s 114us/step - loss: 0.2058 - accuracy: 0.8832 - val_loss: 0.2215 - val_accuracy: 0.8814
Epoch 883/1000
137/137 [==============================] - 0s 114us/step - loss: 0.2059 - accuracy: 0.8832 - val_loss: 0.2234 - val_accuracy: 0.8644
Epoch 884/1000
137/137 [==============================] - 0s 119us/step - loss: 0.2055 - accuracy: 0.8832 - val_loss: 0.2245 - val_accuracy: 0.8644
Epoch 885/1000
137/137 [==============================] - 0s 128us/step - loss: 0.2056 - accuracy: 0.8832 - val_loss: 0.2245 - val_accuracy: 0.8644
Epoch 886/1000
137/137 [==============================] - 0s 133us/step - loss: 0.2056 - accuracy: 0.8832 - val_loss: 0.2261 - val_accuracy: 0.8644
Epoch 887/1000
137/137 [==============================] - 0s 122us/step - loss: 0.2056 - accuracy: 0.8832 - val_loss: 0.2252 - val_accuracy: 0.8644
Epoch 888/1000
137/137 [==============================] - 0s 148us/step - loss: 0.2055 - accuracy: 0.8832 - val_loss: 0.2256 - 

137/137 [==============================] - 0s 107us/step - loss: 0.2082 - accuracy: 0.8832 - val_loss: 0.2282 - val_accuracy: 0.8814
Epoch 938/1000
137/137 [==============================] - 0s 163us/step - loss: 0.2074 - accuracy: 0.8832 - val_loss: 0.2285 - val_accuracy: 0.8814
Epoch 939/1000
137/137 [==============================] - 0s 112us/step - loss: 0.2068 - accuracy: 0.8832 - val_loss: 0.2286 - val_accuracy: 0.8814
Epoch 940/1000
137/137 [==============================] - 0s 108us/step - loss: 0.2064 - accuracy: 0.8832 - val_loss: 0.2284 - val_accuracy: 0.8814
Epoch 941/1000
137/137 [==============================] - 0s 108us/step - loss: 0.2064 - accuracy: 0.8832 - val_loss: 0.2280 - val_accuracy: 0.8814
Epoch 942/1000
137/137 [==============================] - 0s 92us/step - loss: 0.2065 - accuracy: 0.8832 - val_loss: 0.2275 - val_accuracy: 0.8814
Epoch 943/1000
137/137 [==============================] - 0s 107us/step - loss: 0.2059 - accuracy: 0.8832 - val_loss: 0.2244 - v

137/137 [==============================] - 0s 103us/step - loss: 0.2050 - accuracy: 0.8832 - val_loss: 0.2230 - val_accuracy: 0.8814
Epoch 993/1000
137/137 [==============================] - 0s 102us/step - loss: 0.2049 - accuracy: 0.8832 - val_loss: 0.2207 - val_accuracy: 0.8814
Epoch 994/1000
137/137 [==============================] - 0s 108us/step - loss: 0.2050 - accuracy: 0.8832 - val_loss: 0.2252 - val_accuracy: 0.8644
Epoch 995/1000
137/137 [==============================] - 0s 135us/step - loss: 0.2050 - accuracy: 0.8832 - val_loss: 0.2218 - val_accuracy: 0.8644
Epoch 996/1000
137/137 [==============================] - 0s 120us/step - loss: 0.2048 - accuracy: 0.8832 - val_loss: 0.2204 - val_accuracy: 0.8814
Epoch 997/1000
137/137 [==============================] - 0s 132us/step - loss: 0.2048 - accuracy: 0.8832 - val_loss: 0.2210 - val_accuracy: 0.8814
Epoch 998/1000
137/137 [==============================] - 0s 151us/step - loss: 0.2048 - accuracy: 0.8832 - val_loss: 0.2195 - 

In [35]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

59/59 [==============================] - 0s 171us/step
combination test accuracy: 88.14%


In [36]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [37]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 137 samples, validate on 59 samples
Epoch 1/1000
137/137 [==============================] - 0s 2ms/step - loss: 5.1386 - accuracy: 0.5620 - val_loss: 2.4289 - val_accuracy: 0.7627
Epoch 2/1000
137/137 [==============================] - 0s 193us/step - loss: 3.6185 - accuracy: 0.6715 - val_loss: 0.7413 - val_accuracy: 0.6780
Epoch 3/1000
137/137 [==============================] - 0s 169us/step - loss: 2.7564 - accuracy: 0.7518 - val_loss: 0.6890 - val_accuracy: 0.6102
Epoch 4/1000
137/137 [==============================] - 0s 147us/step - loss: 2.0932 - accuracy: 0.7518 - val_loss: 1.0793 - val_accuracy: 0.7966
Epoch 5/1000
137/137 [==============================] - 0s 131us/step - loss: 3.7100 - accuracy: 0.7664 - val_loss: 0.7171 - val_accuracy: 0.6610
Epoch 6/1000
137/137 [==============================] - 0s 101us/step - loss: 2.3834 - accuracy: 0.7810 - val_loss: 0.6182 - val_accuracy: 0.6949
Epoch 7/1000
137/137 [==============================] - 0s 115us/step - loss: 2.0

137/137 [==============================] - 0s 120us/step - loss: 1.2227 - accuracy: 0.8321 - val_loss: 0.4767 - val_accuracy: 0.7797
Epoch 57/1000
137/137 [==============================] - 0s 160us/step - loss: 1.3748 - accuracy: 0.8029 - val_loss: 0.4755 - val_accuracy: 0.7797
Epoch 58/1000
137/137 [==============================] - 0s 114us/step - loss: 1.7955 - accuracy: 0.7883 - val_loss: 0.5233 - val_accuracy: 0.6441
Epoch 59/1000
137/137 [==============================] - 0s 122us/step - loss: 2.1322 - accuracy: 0.7591 - val_loss: 0.5206 - val_accuracy: 0.6441
Epoch 60/1000
137/137 [==============================] - 0s 111us/step - loss: 1.9407 - accuracy: 0.7445 - val_loss: 0.5167 - val_accuracy: 0.6610
Epoch 61/1000
137/137 [==============================] - 0s 104us/step - loss: 1.7044 - accuracy: 0.7664 - val_loss: 0.5107 - val_accuracy: 0.6949
Epoch 62/1000
137/137 [==============================] - 0s 119us/step - loss: 1.2365 - accuracy: 0.8321 - val_loss: 0.4789 - val_ac

Epoch 112/1000
137/137 [==============================] - 0s 112us/step - loss: 1.9986 - accuracy: 0.7737 - val_loss: 0.4561 - val_accuracy: 0.7966
Epoch 113/1000
137/137 [==============================] - 0s 114us/step - loss: 2.2108 - accuracy: 0.7518 - val_loss: 0.4462 - val_accuracy: 0.7797
Epoch 114/1000
137/137 [==============================] - 0s 112us/step - loss: 2.1020 - accuracy: 0.7664 - val_loss: 0.4500 - val_accuracy: 0.7797
Epoch 115/1000
137/137 [==============================] - 0s 175us/step - loss: 1.9896 - accuracy: 0.7810 - val_loss: 0.4386 - val_accuracy: 0.7797
Epoch 116/1000
137/137 [==============================] - 0s 132us/step - loss: 2.2981 - accuracy: 0.7737 - val_loss: 0.4779 - val_accuracy: 0.7119
Epoch 117/1000
137/137 [==============================] - 0s 143us/step - loss: 2.0069 - accuracy: 0.7518 - val_loss: 0.4444 - val_accuracy: 0.7797
Epoch 118/1000
137/137 [==============================] - 0s 147us/step - loss: 2.2211 - accuracy: 0.7445 - val_

137/137 [==============================] - 0s 121us/step - loss: 2.0720 - accuracy: 0.7810 - val_loss: 0.4184 - val_accuracy: 0.7966
Epoch 168/1000
137/137 [==============================] - 0s 116us/step - loss: 2.3989 - accuracy: 0.7664 - val_loss: 0.4175 - val_accuracy: 0.7966
Epoch 169/1000
137/137 [==============================] - 0s 100us/step - loss: 0.9817 - accuracy: 0.8248 - val_loss: 0.4241 - val_accuracy: 0.7966
Epoch 170/1000
137/137 [==============================] - 0s 127us/step - loss: 2.0969 - accuracy: 0.7591 - val_loss: 0.4223 - val_accuracy: 0.7966
Epoch 171/1000
137/137 [==============================] - 0s 122us/step - loss: 1.5309 - accuracy: 0.8029 - val_loss: 0.4233 - val_accuracy: 0.7966
Epoch 172/1000
137/137 [==============================] - 0s 134us/step - loss: 1.3852 - accuracy: 0.8394 - val_loss: 0.4238 - val_accuracy: 0.7966
Epoch 173/1000
137/137 [==============================] - 0s 122us/step - loss: 2.0850 - accuracy: 0.7591 - val_loss: 0.4412 - 

137/137 [==============================] - 0s 120us/step - loss: 1.5285 - accuracy: 0.7956 - val_loss: 0.4465 - val_accuracy: 0.7458
Epoch 223/1000
137/137 [==============================] - 0s 118us/step - loss: 1.0601 - accuracy: 0.8394 - val_loss: 0.4379 - val_accuracy: 0.7627
Epoch 224/1000
137/137 [==============================] - 0s 117us/step - loss: 1.2918 - accuracy: 0.8175 - val_loss: 0.4052 - val_accuracy: 0.8305
Epoch 225/1000
137/137 [==============================] - 0s 113us/step - loss: 1.1858 - accuracy: 0.8175 - val_loss: 0.3949 - val_accuracy: 0.7966
Epoch 226/1000
137/137 [==============================] - 0s 112us/step - loss: 2.2906 - accuracy: 0.7372 - val_loss: 0.3971 - val_accuracy: 0.7966
Epoch 227/1000
137/137 [==============================] - 0s 106us/step - loss: 1.5054 - accuracy: 0.8029 - val_loss: 0.3936 - val_accuracy: 0.7966
Epoch 228/1000
137/137 [==============================] - 0s 111us/step - loss: 1.3832 - accuracy: 0.8175 - val_loss: 0.3988 - 

...

137/137 [==============================] - 0s 143us/step - loss: 1.8003 - accuracy: 0.7883 - val_loss: 0.3348 - val_accuracy: 0.8644
Epoch 773/1000
137/137 [==============================] - 0s 122us/step - loss: 1.6871 - accuracy: 0.8029 - val_loss: 0.3265 - val_accuracy: 0.8814
Epoch 774/1000
137/137 [==============================] - 0s 136us/step - loss: 1.3390 - accuracy: 0.8321 - val_loss: 0.3308 - val_accuracy: 0.8814
Epoch 775/1000
137/137 [==============================] - 0s 116us/step - loss: 2.1441 - accuracy: 0.7591 - val_loss: 0.3310 - val_accuracy: 0.8814
Epoch 776/1000
137/137 [==============================] - 0s 108us/step - loss: 1.9220 - accuracy: 0.7664 - val_loss: 0.3306 - val_accuracy: 0.8814
Epoch 777/1000
137/137 [==============================] - 0s 113us/step - loss: 1.3493 - accuracy: 0.8248 - val_loss: 0.3221 - val_accuracy: 0.8814
Epoch 778/1000
137/137 [==============================] - 0s 115us/step - loss: 1.8195 - accuracy: 0.7810 - val_loss: 0.3274 - 

137/137 [==============================] - 0s 99us/step - loss: 1.5535 - accuracy: 0.8321 - val_loss: 0.3253 - val_accuracy: 0.8814
Epoch 828/1000
137/137 [==============================] - 0s 96us/step - loss: 1.2424 - accuracy: 0.8248 - val_loss: 0.3242 - val_accuracy: 0.8814
Epoch 829/1000
137/137 [==============================] - 0s 109us/step - loss: 1.2507 - accuracy: 0.8248 - val_loss: 0.3308 - val_accuracy: 0.8644
Epoch 830/1000
137/137 [==============================] - 0s 127us/step - loss: 1.5523 - accuracy: 0.8248 - val_loss: 0.3352 - val_accuracy: 0.8644
Epoch 831/1000
137/137 [==============================] - 0s 127us/step - loss: 2.0315 - accuracy: 0.7591 - val_loss: 0.3297 - val_accuracy: 0.8814
Epoch 832/1000
137/137 [==============================] - 0s 117us/step - loss: 2.1138 - accuracy: 0.7883 - val_loss: 0.3134 - val_accuracy: 0.8475
Epoch 833/1000
137/137 [==============================] - 0s 125us/step - loss: 1.2597 - accuracy: 0.8175 - val_loss: 0.3179 - va

137/137 [==============================] - 0s 139us/step - loss: 1.4493 - accuracy: 0.8175 - val_loss: 0.3259 - val_accuracy: 0.8814
Epoch 883/1000
137/137 [==============================] - 0s 128us/step - loss: 2.0331 - accuracy: 0.7737 - val_loss: 0.3237 - val_accuracy: 0.8814
Epoch 884/1000
137/137 [==============================] - 0s 216us/step - loss: 1.6663 - accuracy: 0.8102 - val_loss: 0.3248 - val_accuracy: 0.8814
Epoch 885/1000
137/137 [==============================] - 0s 131us/step - loss: 1.4486 - accuracy: 0.8248 - val_loss: 0.3285 - val_accuracy: 0.8644
Epoch 886/1000
137/137 [==============================] - 0s 205us/step - loss: 1.9117 - accuracy: 0.7737 - val_loss: 0.3346 - val_accuracy: 0.8644
Epoch 887/1000
137/137 [==============================] - 0s 117us/step - loss: 2.1239 - accuracy: 0.7883 - val_loss: 0.3474 - val_accuracy: 0.8644
Epoch 888/1000
137/137 [==============================] - 0s 163us/step - loss: 2.3578 - accuracy: 0.7591 - val_loss: 0.3231 - 

137/137 [==============================] - 0s 171us/step - loss: 1.5831 - accuracy: 0.8029 - val_loss: 0.3225 - val_accuracy: 0.8814
Epoch 938/1000
137/137 [==============================] - 0s 149us/step - loss: 2.1438 - accuracy: 0.7591 - val_loss: 0.3323 - val_accuracy: 0.8644
Epoch 939/1000
137/137 [==============================] - 0s 127us/step - loss: 1.7873 - accuracy: 0.8029 - val_loss: 0.3457 - val_accuracy: 0.8644
Epoch 940/1000
137/137 [==============================] - 0s 137us/step - loss: 1.8072 - accuracy: 0.7810 - val_loss: 0.3197 - val_accuracy: 0.8814
Epoch 941/1000
137/137 [==============================] - 0s 143us/step - loss: 1.6862 - accuracy: 0.7883 - val_loss: 0.3189 - val_accuracy: 0.8814
Epoch 942/1000
137/137 [==============================] - 0s 148us/step - loss: 2.0162 - accuracy: 0.7810 - val_loss: 0.3213 - val_accuracy: 0.8814
Epoch 943/1000
137/137 [==============================] - ETA: 0s - loss: 1.1920 - accuracy: 0.90 - 0s 146us/step - loss: 1.669

137/137 [==============================] - 0s 133us/step - loss: 1.4468 - accuracy: 0.8248 - val_loss: 0.3127 - val_accuracy: 0.8814
Epoch 993/1000
137/137 [==============================] - 0s 115us/step - loss: 1.2505 - accuracy: 0.8175 - val_loss: 0.3263 - val_accuracy: 0.8644
Epoch 994/1000
137/137 [==============================] - 0s 114us/step - loss: 1.3458 - accuracy: 0.8248 - val_loss: 0.3215 - val_accuracy: 0.8814
Epoch 995/1000
137/137 [==============================] - 0s 137us/step - loss: 1.6815 - accuracy: 0.8102 - val_loss: 0.3219 - val_accuracy: 0.8814
Epoch 996/1000
137/137 [==============================] - 0s 113us/step - loss: 1.1472 - accuracy: 0.8102 - val_loss: 0.3193 - val_accuracy: 0.8814
Epoch 997/1000
137/137 [==============================] - 0s 123us/step - loss: 1.6921 - accuracy: 0.7883 - val_loss: 0.3113 - val_accuracy: 0.8814
Epoch 998/1000
137/137 [==============================] - 0s 131us/step - loss: 1.3551 - accuracy: 0.8175 - val_loss: 0.3273 - 

In [39]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

59/59 [==============================] - 0s 108us/step
combination test accuracy: 88.14%


In [40]:
#### improve neural network
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [41]:
model1_comb3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [42]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 137 samples, validate on 59 samples
Epoch 1/1000
137/137 [==============================] - 0s 1ms/step - loss: 4.3366 - accuracy: 0.6131 - val_loss: 0.9000 - val_accuracy: 0.7627
Epoch 2/1000
137/137 [==============================] - 0s 92us/step - loss: 2.0728 - accuracy: 0.6569 - val_loss: 0.7315 - val_accuracy: 0.7966
Epoch 3/1000
137/137 [==============================] - 0s 115us/step - loss: 2.3870 - accuracy: 0.7007 - val_loss: 0.7132 - val_accuracy: 0.8305
Epoch 4/1000
137/137 [==============================] - 0s 136us/step - loss: 2.4399 - accuracy: 0.7518 - val_loss: 0.6864 - val_accuracy: 0.8136
Epoch 5/1000
137/137 [==============================] - 0s 132us/step - loss: 2.2180 - accuracy: 0.7518 - val_loss: 0.6386 - val_accuracy: 0.7966
Epoch 6/1000
137/137 [==============================] - 0s 140us/step - loss: 2.4297 - accuracy: 0.7372 - val_loss: 0.6252 - val_accuracy: 0.7966
Epoch 7/1000
137/137 [==============================] - 0s 122us/step - loss: 2.57

137/137 [==============================] - 0s 136us/step - loss: 1.5831 - accuracy: 0.7956 - val_loss: 0.4632 - val_accuracy: 0.7966
Epoch 57/1000
137/137 [==============================] - 0s 117us/step - loss: 1.2577 - accuracy: 0.8029 - val_loss: 0.4675 - val_accuracy: 0.7966
Epoch 58/1000
137/137 [==============================] - 0s 120us/step - loss: 1.5521 - accuracy: 0.8102 - val_loss: 0.4681 - val_accuracy: 0.7966
Epoch 59/1000
137/137 [==============================] - 0s 118us/step - loss: 1.5915 - accuracy: 0.7810 - val_loss: 0.4852 - val_accuracy: 0.7797
Epoch 60/1000
137/137 [==============================] - 0s 115us/step - loss: 1.1250 - accuracy: 0.8321 - val_loss: 0.4733 - val_accuracy: 0.8305
Epoch 61/1000
137/137 [==============================] - 0s 115us/step - loss: 1.5488 - accuracy: 0.8102 - val_loss: 0.4539 - val_accuracy: 0.7966
Epoch 62/1000
137/137 [==============================] - 0s 114us/step - loss: 1.9016 - accuracy: 0.7883 - val_loss: 0.4522 - val_ac

Epoch 112/1000
137/137 [==============================] - 0s 113us/step - loss: 1.4227 - accuracy: 0.8029 - val_loss: 0.4844 - val_accuracy: 0.6610
Epoch 113/1000
137/137 [==============================] - 0s 102us/step - loss: 1.6707 - accuracy: 0.7956 - val_loss: 0.4830 - val_accuracy: 0.6610
Epoch 114/1000
137/137 [==============================] - 0s 127us/step - loss: 1.3150 - accuracy: 0.8321 - val_loss: 0.4805 - val_accuracy: 0.6610
Epoch 115/1000
137/137 [==============================] - 0s 127us/step - loss: 1.9944 - accuracy: 0.7810 - val_loss: 0.4784 - val_accuracy: 0.6610
Epoch 116/1000
137/137 [==============================] - 0s 125us/step - loss: 1.7544 - accuracy: 0.8029 - val_loss: 0.4772 - val_accuracy: 0.6610
Epoch 117/1000
137/137 [==============================] - 0s 117us/step - loss: 1.1941 - accuracy: 0.8467 - val_loss: 0.4766 - val_accuracy: 0.6610
Epoch 118/1000
137/137 [==============================] - ETA: 0s - loss: 1.7051 - accuracy: 0.87 - 0s 112us/ste

137/137 [==============================] - 0s 130us/step - loss: 1.8647 - accuracy: 0.7956 - val_loss: 0.4100 - val_accuracy: 0.8305
Epoch 168/1000
137/137 [==============================] - 0s 128us/step - loss: 1.5382 - accuracy: 0.8102 - val_loss: 0.4179 - val_accuracy: 0.8305
Epoch 169/1000
137/137 [==============================] - 0s 122us/step - loss: 1.5391 - accuracy: 0.8029 - val_loss: 0.4081 - val_accuracy: 0.8305
Epoch 170/1000
137/137 [==============================] - 0s 120us/step - loss: 2.0865 - accuracy: 0.7810 - val_loss: 0.4113 - val_accuracy: 0.8475
Epoch 171/1000
137/137 [==============================] - 0s 115us/step - loss: 1.6546 - accuracy: 0.7956 - val_loss: 0.4029 - val_accuracy: 0.8475
Epoch 172/1000
137/137 [==============================] - 0s 121us/step - loss: 1.4137 - accuracy: 0.8248 - val_loss: 0.4019 - val_accuracy: 0.8475
Epoch 173/1000
137/137 [==============================] - 0s 122us/step - loss: 1.7605 - accuracy: 0.7956 - val_loss: 0.3986 - 

137/137 [==============================] - 0s 118us/step - loss: 1.4160 - accuracy: 0.8175 - val_loss: 0.3898 - val_accuracy: 0.8475
Epoch 223/1000
137/137 [==============================] - 0s 125us/step - loss: 1.6172 - accuracy: 0.8102 - val_loss: 0.3808 - val_accuracy: 0.8305
Epoch 224/1000
137/137 [==============================] - 0s 139us/step - loss: 1.8376 - accuracy: 0.7956 - val_loss: 0.3826 - val_accuracy: 0.8305
Epoch 225/1000
137/137 [==============================] - 0s 113us/step - loss: 1.4058 - accuracy: 0.8102 - val_loss: 0.3765 - val_accuracy: 0.8305
Epoch 226/1000
137/137 [==============================] - 0s 120us/step - loss: 2.3689 - accuracy: 0.7664 - val_loss: 0.3763 - val_accuracy: 0.8475
Epoch 227/1000
137/137 [==============================] - 0s 112us/step - loss: 2.7409 - accuracy: 0.7226 - val_loss: 0.3786 - val_accuracy: 0.8305
Epoch 228/1000
137/137 [==============================] - 0s 107us/step - loss: 1.2940 - accuracy: 0.8175 - val_loss: 0.3717 - 

...

137/137 [==============================] - 0s 129us/step - loss: 1.8098 - accuracy: 0.7883 - val_loss: 0.3601 - val_accuracy: 0.8644
Epoch 773/1000
137/137 [==============================] - 0s 107us/step - loss: 2.8337 - accuracy: 0.7080 - val_loss: 0.3602 - val_accuracy: 0.8644
Epoch 774/1000
137/137 [==============================] - 0s 105us/step - loss: 1.7077 - accuracy: 0.7956 - val_loss: 0.3609 - val_accuracy: 0.8644
Epoch 775/1000
137/137 [==============================] - 0s 97us/step - loss: 2.1502 - accuracy: 0.7664 - val_loss: 0.3603 - val_accuracy: 0.8644
Epoch 776/1000
137/137 [==============================] - 0s 117us/step - loss: 1.4945 - accuracy: 0.8102 - val_loss: 0.3606 - val_accuracy: 0.8644
Epoch 777/1000
137/137 [==============================] - 0s 105us/step - loss: 1.3755 - accuracy: 0.8175 - val_loss: 0.3603 - val_accuracy: 0.8644
Epoch 778/1000
137/137 [==============================] - 0s 123us/step - loss: 1.3754 - accuracy: 0.8175 - val_loss: 0.3602 - v

137/137 [==============================] - 0s 115us/step - loss: 1.8204 - accuracy: 0.7883 - val_loss: 0.3571 - val_accuracy: 0.8644
Epoch 828/1000
137/137 [==============================] - 0s 132us/step - loss: 1.5638 - accuracy: 0.8321 - val_loss: 0.3566 - val_accuracy: 0.8644
Epoch 829/1000
137/137 [==============================] - 0s 116us/step - loss: 1.8206 - accuracy: 0.7810 - val_loss: 0.3568 - val_accuracy: 0.8644
Epoch 830/1000
137/137 [==============================] - 0s 125us/step - loss: 1.3889 - accuracy: 0.8029 - val_loss: 0.3543 - val_accuracy: 0.8644
Epoch 831/1000
137/137 [==============================] - 0s 114us/step - loss: 2.3266 - accuracy: 0.7883 - val_loss: 0.3547 - val_accuracy: 0.8644
Epoch 832/1000
137/137 [==============================] - 0s 107us/step - loss: 1.4690 - accuracy: 0.8248 - val_loss: 0.3552 - val_accuracy: 0.8644
Epoch 833/1000
137/137 [==============================] - 0s 115us/step - loss: 1.3697 - accuracy: 0.8175 - val_loss: 0.3567 - 

137/137 [==============================] - 0s 123us/step - loss: 1.3202 - accuracy: 0.8613 - val_loss: 0.3528 - val_accuracy: 0.8644
Epoch 883/1000
137/137 [==============================] - 0s 127us/step - loss: 1.7241 - accuracy: 0.7810 - val_loss: 0.3519 - val_accuracy: 0.8644
Epoch 884/1000
137/137 [==============================] - 0s 128us/step - loss: 1.3716 - accuracy: 0.8175 - val_loss: 0.3526 - val_accuracy: 0.8644
Epoch 885/1000
137/137 [==============================] - 0s 135us/step - loss: 1.1268 - accuracy: 0.8540 - val_loss: 0.3521 - val_accuracy: 0.8644
Epoch 886/1000
137/137 [==============================] - 0s 139us/step - loss: 1.2527 - accuracy: 0.8321 - val_loss: 0.3518 - val_accuracy: 0.8644
Epoch 887/1000
137/137 [==============================] - 0s 133us/step - loss: 2.0299 - accuracy: 0.7810 - val_loss: 0.3526 - val_accuracy: 0.8644
Epoch 888/1000
137/137 [==============================] - 0s 107us/step - loss: 1.8243 - accuracy: 0.7883 - val_loss: 0.3512 - 

137/137 [==============================] - 0s 125us/step - loss: 2.0206 - accuracy: 0.7810 - val_loss: 0.3575 - val_accuracy: 0.8644
Epoch 938/1000
137/137 [==============================] - 0s 120us/step - loss: 2.0376 - accuracy: 0.7737 - val_loss: 0.3585 - val_accuracy: 0.8644
Epoch 939/1000
137/137 [==============================] - 0s 123us/step - loss: 1.7092 - accuracy: 0.7956 - val_loss: 0.3585 - val_accuracy: 0.8644
Epoch 940/1000
137/137 [==============================] - 0s 121us/step - loss: 1.0528 - accuracy: 0.8248 - val_loss: 0.3580 - val_accuracy: 0.8644
Epoch 941/1000
137/137 [==============================] - 0s 113us/step - loss: 2.2741 - accuracy: 0.7518 - val_loss: 0.3579 - val_accuracy: 0.8644
Epoch 942/1000
137/137 [==============================] - 0s 120us/step - loss: 1.5815 - accuracy: 0.8102 - val_loss: 0.3566 - val_accuracy: 0.8644
Epoch 943/1000
137/137 [==============================] - 0s 121us/step - loss: 1.7117 - accuracy: 0.7883 - val_loss: 0.3583 - 

137/137 [==============================] - 0s 132us/step - loss: 1.9081 - accuracy: 0.7956 - val_loss: 0.3586 - val_accuracy: 0.8644
Epoch 993/1000
137/137 [==============================] - 0s 130us/step - loss: 1.3746 - accuracy: 0.8102 - val_loss: 0.3574 - val_accuracy: 0.8644
Epoch 994/1000
137/137 [==============================] - 0s 108us/step - loss: 1.6939 - accuracy: 0.8029 - val_loss: 0.3579 - val_accuracy: 0.8644
Epoch 995/1000
137/137 [==============================] - 0s 113us/step - loss: 2.0229 - accuracy: 0.7810 - val_loss: 0.3577 - val_accuracy: 0.8644
Epoch 996/1000
137/137 [==============================] - 0s 110us/step - loss: 1.3802 - accuracy: 0.8029 - val_loss: 0.3576 - val_accuracy: 0.8644
Epoch 997/1000
137/137 [==============================] - 0s 109us/step - loss: 1.1433 - accuracy: 0.8394 - val_loss: 0.3576 - val_accuracy: 0.8644
Epoch 998/1000
137/137 [==============================] - 0s 116us/step - loss: 1.6913 - accuracy: 0.8029 - val_loss: 0.3574 - 

In [43]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

59/59 [==============================] - 0s 116us/step
combination test accuracy: 86.44%


In [44]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [45]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 479us/step - loss: 11.5243 - accuracy: 0.6061 - val_loss: 0.6200 - val_accuracy: 0.6970
Epoch 2/1000
231/231 [==============================] - 0s 71us/step - loss: 0.6300 - accuracy: 0.7013 - val_loss: 0.6055 - val_accuracy: 0.6970
Epoch 3/1000
231/231 [==============================] - 0s 89us/step - loss: 0.6181 - accuracy: 0.6970 - val_loss: 0.5988 - val_accuracy: 0.7172
Epoch 4/1000
231/231 [==============================] - 0s 109us/step - loss: 0.6016 - accuracy: 0.7143 - val_loss: 0.5805 - val_accuracy: 0.7475
Epoch 5/1000
231/231 [==============================] - 0s 96us/step - loss: 0.6181 - accuracy: 0.7056 - val_loss: 0.5864 - val_accuracy: 0.7273
Epoch 6/1000
231/231 [==============================] - 0s 92us/step - loss: 0.6112 - accuracy: 0.6840 - val_loss: 0.5778 - val_accuracy: 0.7374
Epoch 7/1000
231/231 [==============================] - 0s 93us/step - loss: 0.583

Epoch 57/1000
231/231 [==============================] - 0s 96us/step - loss: 0.4983 - accuracy: 0.7229 - val_loss: 0.5921 - val_accuracy: 0.7374
Epoch 58/1000
231/231 [==============================] - 0s 91us/step - loss: 0.5121 - accuracy: 0.7359 - val_loss: 0.5674 - val_accuracy: 0.6768
Epoch 59/1000
231/231 [==============================] - 0s 98us/step - loss: 0.5181 - accuracy: 0.7186 - val_loss: 0.5233 - val_accuracy: 0.7273
Epoch 60/1000
231/231 [==============================] - 0s 97us/step - loss: 0.4845 - accuracy: 0.7403 - val_loss: 0.5265 - val_accuracy: 0.7071
Epoch 61/1000
231/231 [==============================] - 0s 87us/step - loss: 0.4955 - accuracy: 0.7359 - val_loss: 0.5128 - val_accuracy: 0.7273
Epoch 62/1000
231/231 [==============================] - 0s 90us/step - loss: 0.5042 - accuracy: 0.7316 - val_loss: 0.5228 - val_accuracy: 0.7172
Epoch 63/1000
231/231 [==============================] - 0s 85us/step - loss: 0.4863 - accuracy: 0.7446 - val_loss: 0.5007 -

231/231 [==============================] - 0s 94us/step - loss: 0.4569 - accuracy: 0.7576 - val_loss: 0.5087 - val_accuracy: 0.7374
Epoch 114/1000
231/231 [==============================] - 0s 114us/step - loss: 0.4670 - accuracy: 0.7532 - val_loss: 0.4955 - val_accuracy: 0.7475
Epoch 115/1000
231/231 [==============================] - 0s 94us/step - loss: 0.4467 - accuracy: 0.7792 - val_loss: 0.5065 - val_accuracy: 0.7374
Epoch 116/1000
231/231 [==============================] - 0s 83us/step - loss: 0.4433 - accuracy: 0.7706 - val_loss: 0.4967 - val_accuracy: 0.7273
Epoch 117/1000
231/231 [==============================] - 0s 85us/step - loss: 0.4776 - accuracy: 0.7619 - val_loss: 0.4860 - val_accuracy: 0.7677
Epoch 118/1000
231/231 [==============================] - 0s 90us/step - loss: 0.4485 - accuracy: 0.7792 - val_loss: 0.4945 - val_accuracy: 0.7273
Epoch 119/1000
231/231 [==============================] - 0s 87us/step - loss: 0.4620 - accuracy: 0.7446 - val_loss: 0.5433 - val_ac

231/231 [==============================] - 0s 89us/step - loss: 0.4570 - accuracy: 0.7706 - val_loss: 0.4988 - val_accuracy: 0.7677
Epoch 169/1000
231/231 [==============================] - 0s 86us/step - loss: 0.4410 - accuracy: 0.7792 - val_loss: 0.4943 - val_accuracy: 0.7778
Epoch 170/1000
231/231 [==============================] - 0s 92us/step - loss: 0.4384 - accuracy: 0.7792 - val_loss: 0.4913 - val_accuracy: 0.7677
Epoch 171/1000
231/231 [==============================] - 0s 91us/step - loss: 0.4316 - accuracy: 0.7792 - val_loss: 0.5051 - val_accuracy: 0.7576
Epoch 172/1000
231/231 [==============================] - 0s 91us/step - loss: 0.4420 - accuracy: 0.7879 - val_loss: 0.4904 - val_accuracy: 0.7879
Epoch 173/1000
231/231 [==============================] - 0s 96us/step - loss: 0.4310 - accuracy: 0.7749 - val_loss: 0.4881 - val_accuracy: 0.7576
Epoch 174/1000
231/231 [==============================] - 0s 83us/step - loss: 0.4339 - accuracy: 0.7792 - val_loss: 0.5029 - val_acc

Epoch 224/1000
231/231 [==============================] - 0s 90us/step - loss: 0.4205 - accuracy: 0.7922 - val_loss: 0.5136 - val_accuracy: 0.7677
Epoch 225/1000
231/231 [==============================] - 0s 90us/step - loss: 0.4083 - accuracy: 0.8052 - val_loss: 0.4795 - val_accuracy: 0.7576
Epoch 226/1000
231/231 [==============================] - 0s 87us/step - loss: 0.3978 - accuracy: 0.8009 - val_loss: 0.4824 - val_accuracy: 0.7677
Epoch 227/1000
231/231 [==============================] - 0s 88us/step - loss: 0.3979 - accuracy: 0.7922 - val_loss: 0.4755 - val_accuracy: 0.7677
Epoch 228/1000
231/231 [==============================] - 0s 87us/step - loss: 0.3926 - accuracy: 0.8095 - val_loss: 0.4968 - val_accuracy: 0.7677
Epoch 229/1000
231/231 [==============================] - 0s 99us/step - loss: 0.3954 - accuracy: 0.8009 - val_loss: 0.4875 - val_accuracy: 0.7677
Epoch 230/1000
231/231 [==============================] - 0s 89us/step - loss: 0.4706 - accuracy: 0.7879 - val_loss: 0

...

Epoch 726/1000
231/231 [==============================] - 0s 91us/step - loss: 0.2834 - accuracy: 0.8571 - val_loss: 0.5385 - val_accuracy: 0.7677
Epoch 727/1000
231/231 [==============================] - 0s 87us/step - loss: 0.2896 - accuracy: 0.8485 - val_loss: 0.5587 - val_accuracy: 0.7677
Epoch 728/1000
231/231 [==============================] - 0s 85us/step - loss: 0.2900 - accuracy: 0.8528 - val_loss: 0.5220 - val_accuracy: 0.7778
Epoch 729/1000
231/231 [==============================] - 0s 94us/step - loss: 0.3052 - accuracy: 0.8485 - val_loss: 0.5119 - val_accuracy: 0.7677
Epoch 730/1000
231/231 [==============================] - 0s 89us/step - loss: 0.2896 - accuracy: 0.8528 - val_loss: 0.4927 - val_accuracy: 0.7879
Epoch 731/1000
231/231 [==============================] - 0s 83us/step - loss: 0.3072 - accuracy: 0.8442 - val_loss: 0.5517 - val_accuracy: 0.7879
Epoch 732/1000
231/231 [==============================] - 0s 96us/step - loss: 0.3049 - accuracy: 0.8398 - val_loss: 0

Epoch 782/1000
231/231 [==============================] - 0s 94us/step - loss: 0.3066 - accuracy: 0.8312 - val_loss: 0.4987 - val_accuracy: 0.7778
Epoch 783/1000
231/231 [==============================] - 0s 112us/step - loss: 0.3080 - accuracy: 0.8312 - val_loss: 0.4982 - val_accuracy: 0.7778
Epoch 784/1000
231/231 [==============================] - 0s 130us/step - loss: 0.3076 - accuracy: 0.8225 - val_loss: 0.5913 - val_accuracy: 0.7677
Epoch 785/1000
231/231 [==============================] - 0s 166us/step - loss: 0.3387 - accuracy: 0.8182 - val_loss: 0.5044 - val_accuracy: 0.7980
Epoch 786/1000
231/231 [==============================] - 0s 96us/step - loss: 0.3131 - accuracy: 0.8398 - val_loss: 0.5020 - val_accuracy: 0.7576
Epoch 787/1000
231/231 [==============================] - 0s 93us/step - loss: 0.3144 - accuracy: 0.8182 - val_loss: 0.5326 - val_accuracy: 0.7879
Epoch 788/1000
231/231 [==============================] - 0s 86us/step - loss: 0.3129 - accuracy: 0.8442 - val_loss

231/231 [==============================] - 0s 87us/step - loss: 0.4824 - accuracy: 0.8398 - val_loss: 0.4972 - val_accuracy: 0.8081
Epoch 838/1000
231/231 [==============================] - 0s 82us/step - loss: 0.2808 - accuracy: 0.8528 - val_loss: 0.5030 - val_accuracy: 0.8081
Epoch 839/1000
231/231 [==============================] - 0s 85us/step - loss: 0.2864 - accuracy: 0.8528 - val_loss: 0.4968 - val_accuracy: 0.7980
Epoch 840/1000
231/231 [==============================] - 0s 92us/step - loss: 0.2822 - accuracy: 0.8615 - val_loss: 0.4929 - val_accuracy: 0.7879
Epoch 841/1000
231/231 [==============================] - 0s 95us/step - loss: 0.2851 - accuracy: 0.8485 - val_loss: 0.5317 - val_accuracy: 0.7980
Epoch 842/1000
231/231 [==============================] - 0s 88us/step - loss: 0.2849 - accuracy: 0.8485 - val_loss: 0.5163 - val_accuracy: 0.7677
Epoch 843/1000
231/231 [==============================] - 0s 87us/step - loss: 0.3052 - accuracy: 0.8355 - val_loss: 0.4929 - val_acc

Epoch 893/1000
231/231 [==============================] - 0s 90us/step - loss: 0.2673 - accuracy: 0.8615 - val_loss: 0.5033 - val_accuracy: 0.7980
Epoch 894/1000
231/231 [==============================] - 0s 92us/step - loss: 0.2752 - accuracy: 0.8615 - val_loss: 0.5137 - val_accuracy: 0.7778
Epoch 895/1000
231/231 [==============================] - 0s 92us/step - loss: 0.2654 - accuracy: 0.8615 - val_loss: 0.5136 - val_accuracy: 0.7980
Epoch 896/1000
231/231 [==============================] - 0s 91us/step - loss: 0.2913 - accuracy: 0.8528 - val_loss: 0.5616 - val_accuracy: 0.7576
Epoch 897/1000
231/231 [==============================] - 0s 99us/step - loss: 0.2711 - accuracy: 0.8571 - val_loss: 0.5078 - val_accuracy: 0.7879
Epoch 898/1000
231/231 [==============================] - 0s 101us/step - loss: 0.2837 - accuracy: 0.8528 - val_loss: 0.5001 - val_accuracy: 0.7980
Epoch 899/1000
231/231 [==============================] - 0s 90us/step - loss: 0.2734 - accuracy: 0.8528 - val_loss: 

231/231 [==============================] - 0s 83us/step - loss: 0.2666 - accuracy: 0.8658 - val_loss: 0.5236 - val_accuracy: 0.7778
Epoch 949/1000
231/231 [==============================] - 0s 93us/step - loss: 0.2632 - accuracy: 0.8571 - val_loss: 0.7027 - val_accuracy: 0.7475
Epoch 950/1000
231/231 [==============================] - 0s 94us/step - loss: 0.4053 - accuracy: 0.8182 - val_loss: 0.5271 - val_accuracy: 0.7879
Epoch 951/1000
231/231 [==============================] - 0s 87us/step - loss: 0.2910 - accuracy: 0.8442 - val_loss: 0.5215 - val_accuracy: 0.7778
Epoch 952/1000
231/231 [==============================] - 0s 88us/step - loss: 0.2888 - accuracy: 0.8442 - val_loss: 0.5347 - val_accuracy: 0.7677
Epoch 953/1000
231/231 [==============================] - 0s 100us/step - loss: 0.2793 - accuracy: 0.8571 - val_loss: 0.5201 - val_accuracy: 0.7879
Epoch 954/1000
231/231 [==============================] - 0s 91us/step - loss: 0.2671 - accuracy: 0.8571 - val_loss: 0.5058 - val_ac

In [47]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

99/99 [==============================] - 0s 85us/step
over-sampling test accuracy: 77.78%


In [115]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [116]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [117]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 536us/step - loss: 3.9740 - accuracy: 0.4848 - val_loss: 1.0847 - val_accuracy: 0.7172
Epoch 2/1000
231/231 [==============================] - 0s 57us/step - loss: 4.8068 - accuracy: 0.5368 - val_loss: 0.6672 - val_accuracy: 0.6667
Epoch 3/1000
231/231 [==============================] - 0s 69us/step - loss: 0.6676 - accuracy: 0.6623 - val_loss: 0.6913 - val_accuracy: 0.6667
Epoch 4/1000
231/231 [==============================] - 0s 72us/step - loss: 0.6518 - accuracy: 0.6797 - val_loss: 0.6492 - val_accuracy: 0.7374
Epoch 5/1000
231/231 [==============================] - 0s 79us/step - loss: 0.6502 - accuracy: 0.7100 - val_loss: 0.6397 - val_accuracy: 0.7677
Epoch 6/1000
231/231 [==============================] - 0s 76us/step - loss: 0.6432 - accuracy: 0.6883 - val_loss: 0.6319 - val_accuracy: 0.7677
Epoch 7/1000
231/231 [==============================] - 0s 81us/step - loss: 0.6370 

Epoch 57/1000
231/231 [==============================] - 0s 85us/step - loss: 0.5249 - accuracy: 0.7316 - val_loss: 0.5261 - val_accuracy: 0.7576
Epoch 58/1000
231/231 [==============================] - 0s 68us/step - loss: 0.5226 - accuracy: 0.7273 - val_loss: 0.5259 - val_accuracy: 0.7576
Epoch 59/1000
231/231 [==============================] - 0s 82us/step - loss: 0.5212 - accuracy: 0.7273 - val_loss: 0.5246 - val_accuracy: 0.7576
Epoch 60/1000
231/231 [==============================] - 0s 71us/step - loss: 0.5212 - accuracy: 0.7316 - val_loss: 0.5242 - val_accuracy: 0.7475
Epoch 61/1000
231/231 [==============================] - 0s 74us/step - loss: 0.5209 - accuracy: 0.7316 - val_loss: 0.5257 - val_accuracy: 0.7475
Epoch 62/1000
231/231 [==============================] - 0s 245us/step - loss: 0.5224 - accuracy: 0.7316 - val_loss: 0.5232 - val_accuracy: 0.7475
Epoch 63/1000
231/231 [==============================] - 0s 252us/step - loss: 0.5160 - accuracy: 0.7446 - val_loss: 0.5212

Epoch 113/1000
231/231 [==============================] - 0s 100us/step - loss: 0.4879 - accuracy: 0.7749 - val_loss: 0.5081 - val_accuracy: 0.7778
Epoch 114/1000
231/231 [==============================] - 0s 81us/step - loss: 0.4850 - accuracy: 0.7749 - val_loss: 0.5091 - val_accuracy: 0.7879
Epoch 115/1000
231/231 [==============================] - 0s 91us/step - loss: 0.4840 - accuracy: 0.7749 - val_loss: 0.5070 - val_accuracy: 0.7677
Epoch 116/1000
231/231 [==============================] - 0s 71us/step - loss: 0.4888 - accuracy: 0.7662 - val_loss: 0.5064 - val_accuracy: 0.7677
Epoch 117/1000
231/231 [==============================] - 0s 114us/step - loss: 0.4829 - accuracy: 0.7792 - val_loss: 0.5062 - val_accuracy: 0.7677
Epoch 118/1000
231/231 [==============================] - 0s 178us/step - loss: 0.4869 - accuracy: 0.7619 - val_loss: 0.5092 - val_accuracy: 0.7879
Epoch 119/1000
231/231 [==============================] - 0s 96us/step - loss: 0.4818 - accuracy: 0.7879 - val_loss

231/231 [==============================] - 0s 66us/step - loss: 0.4586 - accuracy: 0.7879 - val_loss: 0.5043 - val_accuracy: 0.7576
Epoch 169/1000
231/231 [==============================] - 0s 69us/step - loss: 0.5502 - accuracy: 0.7446 - val_loss: 0.5080 - val_accuracy: 0.7778
Epoch 170/1000
231/231 [==============================] - 0s 59us/step - loss: 0.4702 - accuracy: 0.7879 - val_loss: 0.4948 - val_accuracy: 0.7879
Epoch 171/1000
231/231 [==============================] - 0s 63us/step - loss: 0.4587 - accuracy: 0.7879 - val_loss: 0.4913 - val_accuracy: 0.7879
Epoch 172/1000
231/231 [==============================] - 0s 73us/step - loss: 0.4569 - accuracy: 0.7879 - val_loss: 0.4896 - val_accuracy: 0.7778
Epoch 173/1000
231/231 [==============================] - 0s 67us/step - loss: 0.4548 - accuracy: 0.7879 - val_loss: 0.4892 - val_accuracy: 0.7879
Epoch 174/1000
231/231 [==============================] - 0s 68us/step - loss: 0.4556 - accuracy: 0.7749 - val_loss: 0.4900 - val_acc

231/231 [==============================] - 0s 91us/step - loss: 0.4354 - accuracy: 0.7965 - val_loss: 0.4793 - val_accuracy: 0.7879
Epoch 224/1000
231/231 [==============================] - 0s 85us/step - loss: 0.4335 - accuracy: 0.7965 - val_loss: 0.4790 - val_accuracy: 0.7879
Epoch 225/1000
231/231 [==============================] - 0s 116us/step - loss: 0.4336 - accuracy: 0.7922 - val_loss: 0.4841 - val_accuracy: 0.7879
Epoch 226/1000
231/231 [==============================] - 0s 149us/step - loss: 0.4367 - accuracy: 0.7922 - val_loss: 0.4788 - val_accuracy: 0.7879
Epoch 227/1000
231/231 [==============================] - 0s 144us/step - loss: 0.4339 - accuracy: 0.8009 - val_loss: 0.4790 - val_accuracy: 0.7879
Epoch 228/1000
231/231 [==============================] - 0s 70us/step - loss: 0.4329 - accuracy: 0.8009 - val_loss: 0.4786 - val_accuracy: 0.7879
Epoch 229/1000
231/231 [==============================] - 0s 236us/step - loss: 0.4306 - accuracy: 0.7965 - val_loss: 0.4783 - val

...

231/231 [==============================] - 0s 81us/step - loss: 0.3386 - accuracy: 0.8139 - val_loss: 0.5167 - val_accuracy: 0.7879
Epoch 723/1000
231/231 [==============================] - 0s 67us/step - loss: 0.3203 - accuracy: 0.8355 - val_loss: 0.5133 - val_accuracy: 0.7677
Epoch 724/1000
231/231 [==============================] - 0s 74us/step - loss: 0.3098 - accuracy: 0.8442 - val_loss: 0.5494 - val_accuracy: 0.7677
Epoch 725/1000
231/231 [==============================] - 0s 68us/step - loss: 0.3141 - accuracy: 0.8485 - val_loss: 0.5112 - val_accuracy: 0.7980
Epoch 726/1000
231/231 [==============================] - 0s 86us/step - loss: 0.3064 - accuracy: 0.8485 - val_loss: 0.5202 - val_accuracy: 0.7778
Epoch 727/1000
231/231 [==============================] - 0s 69us/step - loss: 0.3083 - accuracy: 0.8442 - val_loss: 0.5128 - val_accuracy: 0.7980
Epoch 728/1000
231/231 [==============================] - 0s 63us/step - loss: 0.3026 - accuracy: 0.8528 - val_loss: 0.5283 - val_acc

Epoch 778/1000
231/231 [==============================] - 0s 63us/step - loss: 0.2972 - accuracy: 0.8571 - val_loss: 0.5777 - val_accuracy: 0.7778
Epoch 779/1000
231/231 [==============================] - 0s 75us/step - loss: 0.3388 - accuracy: 0.8139 - val_loss: 0.5254 - val_accuracy: 0.7778
Epoch 780/1000
231/231 [==============================] - 0s 77us/step - loss: 0.3033 - accuracy: 0.8398 - val_loss: 0.5226 - val_accuracy: 0.7980
Epoch 781/1000
231/231 [==============================] - 0s 71us/step - loss: 0.2973 - accuracy: 0.8485 - val_loss: 0.5248 - val_accuracy: 0.7879
Epoch 782/1000
231/231 [==============================] - 0s 78us/step - loss: 0.3355 - accuracy: 0.8225 - val_loss: 0.5198 - val_accuracy: 0.7980
Epoch 783/1000
231/231 [==============================] - 0s 51us/step - loss: 0.2983 - accuracy: 0.8485 - val_loss: 0.5317 - val_accuracy: 0.7778
Epoch 784/1000
231/231 [==============================] - 0s 89us/step - loss: 0.3063 - accuracy: 0.8398 - val_loss: 0

Epoch 834/1000
231/231 [==============================] - 0s 63us/step - loss: 0.2911 - accuracy: 0.8528 - val_loss: 0.6998 - val_accuracy: 0.7576
Epoch 835/1000
231/231 [==============================] - 0s 71us/step - loss: 0.4345 - accuracy: 0.7792 - val_loss: 0.5456 - val_accuracy: 0.7879
Epoch 836/1000
231/231 [==============================] - 0s 79us/step - loss: 0.3116 - accuracy: 0.8485 - val_loss: 0.5393 - val_accuracy: 0.7778
Epoch 837/1000
231/231 [==============================] - 0s 96us/step - loss: 0.2957 - accuracy: 0.8528 - val_loss: 0.5272 - val_accuracy: 0.7980
Epoch 838/1000
231/231 [==============================] - 0s 71us/step - loss: 0.2944 - accuracy: 0.8355 - val_loss: 0.5262 - val_accuracy: 0.7980
Epoch 839/1000
231/231 [==============================] - 0s 73us/step - loss: 0.2889 - accuracy: 0.8528 - val_loss: 0.5252 - val_accuracy: 0.7980
Epoch 840/1000
231/231 [==============================] - 0s 63us/step - loss: 0.3105 - accuracy: 0.8485 - val_loss: 0

Epoch 890/1000
231/231 [==============================] - 0s 77us/step - loss: 0.2870 - accuracy: 0.8442 - val_loss: 0.5403 - val_accuracy: 0.7879
Epoch 891/1000
231/231 [==============================] - 0s 80us/step - loss: 0.2967 - accuracy: 0.8398 - val_loss: 0.5399 - val_accuracy: 0.7980
Epoch 892/1000
231/231 [==============================] - 0s 73us/step - loss: 0.2861 - accuracy: 0.8571 - val_loss: 0.5840 - val_accuracy: 0.7879
Epoch 893/1000
231/231 [==============================] - 0s 82us/step - loss: 0.3211 - accuracy: 0.8442 - val_loss: 0.5661 - val_accuracy: 0.7677
Epoch 894/1000
231/231 [==============================] - 0s 76us/step - loss: 0.3169 - accuracy: 0.8312 - val_loss: 0.5740 - val_accuracy: 0.7677
Epoch 895/1000
231/231 [==============================] - 0s 99us/step - loss: 0.3132 - accuracy: 0.8268 - val_loss: 0.5393 - val_accuracy: 0.7980
Epoch 896/1000
231/231 [==============================] - 0s 70us/step - loss: 0.2777 - accuracy: 0.8615 - val_loss: 0

Epoch 946/1000
231/231 [==============================] - 0s 65us/step - loss: 0.2861 - accuracy: 0.8442 - val_loss: 0.5620 - val_accuracy: 0.7778
Epoch 947/1000
231/231 [==============================] - 0s 68us/step - loss: 0.2745 - accuracy: 0.8571 - val_loss: 0.5569 - val_accuracy: 0.7778
Epoch 948/1000
231/231 [==============================] - 0s 66us/step - loss: 0.2771 - accuracy: 0.8485 - val_loss: 0.5683 - val_accuracy: 0.7677
Epoch 949/1000
231/231 [==============================] - 0s 61us/step - loss: 0.2951 - accuracy: 0.8355 - val_loss: 0.5560 - val_accuracy: 0.7879
Epoch 950/1000
231/231 [==============================] - 0s 73us/step - loss: 0.2790 - accuracy: 0.8528 - val_loss: 0.5734 - val_accuracy: 0.7677
Epoch 951/1000
231/231 [==============================] - 0s 71us/step - loss: 0.2702 - accuracy: 0.8571 - val_loss: 0.5472 - val_accuracy: 0.8081
Epoch 952/1000
231/231 [==============================] - 0s 71us/step - loss: 0.2722 - accuracy: 0.8571 - val_loss: 0

In [118]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

99/99 [==============================] - 0s 103us/step
over-sampling test accuracy: 79.80%


In [52]:
############ Feature selection using lasso ##########

In [53]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [54]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [55]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [56]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [57]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 373
selected features: 26


In [58]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  0,  30,  64,  77,  81,  83,  93,  95, 117, 118, 124, 152, 178,
        189, 203, 209, 212, 213, 222, 224, 226, 250, 296, 335, 371, 372]])

In [59]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT',
       'TTATGTTTCTGGA', 'TCGTGTCTGT', 'TCACACCGCCT', 'TATCAGTTTTGTT',
       'TAGATTCAAATAT',
       'TAACAAAACTGATAATTAATATACTGCTTATCAATAACATTAAGCGGTGGTGAAACTTCATCATT',
       'TAAATCGTCTTTATATTTAATTATTAAATTAACAAATTTTAATTGGCGGATGAGGTATCCAGTTACCTCGTTCGCCAATTATTTTTCGCAATATAAAAAG',
       'GTGAATTCATG', 'GTCTGATTTTTT', 'GTAGTAGATAATTTT', 'CTAATCCTTCAAT',
       'ACTTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTT',
       'X1_725392_T_A', 'fnbA', 'group_6584', 'scc', 'group_662',
       'yezG_1', 'group_3025', 'group_4981', 'group_4761', 'group_7718',
       'group_2541', 'ST', 'CC'], dtype='<U100')

In [60]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            361, 362, 363, 364, 365, 366, 367, 368, 369, 370],
           dtype='int64', length=347)

In [61]:
len(removed_features)

347

In [62]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 26) (255,) (255, 27)


In [63]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 106), (1, 89)]


In [64]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 165), (1, 165)]


In [65]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel)

In [66]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [67]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [68]:
#### neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [69]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [70]:
model2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 590us/step - loss: 51.1967 - accuracy: 0.5281 - val_loss: 0.6901 - val_accuracy: 0.5714
Epoch 2/1000
178/178 [==============================] - 0s 73us/step - loss: 0.6755 - accuracy: 0.6404 - val_loss: 0.6860 - val_accuracy: 0.5974
Epoch 3/1000
178/178 [==============================] - 0s 79us/step - loss: 0.6694 - accuracy: 0.6517 - val_loss: 0.7343 - val_accuracy: 0.5844
Epoch 4/1000
178/178 [==============================] - 0s 88us/step - loss: 0.6651 - accuracy: 0.6910 - val_loss: 0.6810 - val_accuracy: 0.6234
Epoch 5/1000
178/178 [==============================] - 0s 96us/step - loss: 0.6582 - accuracy: 0.6573 - val_loss: 0.6846 - val_accuracy: 0.6234
Epoch 6/1000
178/178 [==============================] - 0s 91us/step - loss: 0.6638 - accuracy: 0.6685 - val_loss: 0.6829 - val_accuracy: 0.6234
Epoch 7/1000
178/178 [==============================] - 0s 109us/step - loss: 0.678

Epoch 57/1000
178/178 [==============================] - 0s 94us/step - loss: 0.5845 - accuracy: 0.7528 - val_loss: 0.6291 - val_accuracy: 0.7013
Epoch 58/1000
178/178 [==============================] - 0s 82us/step - loss: 0.5630 - accuracy: 0.7640 - val_loss: 0.6333 - val_accuracy: 0.7143
Epoch 59/1000
178/178 [==============================] - 0s 93us/step - loss: 0.5178 - accuracy: 0.8034 - val_loss: 0.6550 - val_accuracy: 0.7013
Epoch 60/1000
178/178 [==============================] - 0s 79us/step - loss: 0.5675 - accuracy: 0.7865 - val_loss: 1.0910 - val_accuracy: 0.5974
Epoch 61/1000
178/178 [==============================] - 0s 84us/step - loss: 0.6218 - accuracy: 0.7360 - val_loss: 0.6560 - val_accuracy: 0.7013
Epoch 62/1000
178/178 [==============================] - 0s 95us/step - loss: 0.5133 - accuracy: 0.8090 - val_loss: 0.6386 - val_accuracy: 0.7143
Epoch 63/1000
178/178 [==============================] - 0s 82us/step - loss: 0.5228 - accuracy: 0.8090 - val_loss: 0.6347 -

Epoch 113/1000
178/178 [==============================] - 0s 116us/step - loss: 0.5333 - accuracy: 0.8090 - val_loss: 0.6535 - val_accuracy: 0.7273
Epoch 114/1000
178/178 [==============================] - 0s 147us/step - loss: 0.4843 - accuracy: 0.8090 - val_loss: 0.7092 - val_accuracy: 0.7143
Epoch 115/1000
178/178 [==============================] - 0s 140us/step - loss: 0.5225 - accuracy: 0.7978 - val_loss: 0.6601 - val_accuracy: 0.7273
Epoch 116/1000
178/178 [==============================] - 0s 123us/step - loss: 0.4880 - accuracy: 0.8258 - val_loss: 0.6490 - val_accuracy: 0.7273
Epoch 117/1000
178/178 [==============================] - 0s 104us/step - loss: 0.5116 - accuracy: 0.8202 - val_loss: 0.7405 - val_accuracy: 0.6623
Epoch 118/1000
178/178 [==============================] - 0s 92us/step - loss: 0.5729 - accuracy: 0.7528 - val_loss: 0.6428 - val_accuracy: 0.7273
Epoch 119/1000
178/178 [==============================] - 0s 97us/step - loss: 0.5626 - accuracy: 0.7809 - val_lo

178/178 [==============================] - 0s 99us/step - loss: 0.4666 - accuracy: 0.8146 - val_loss: 0.6088 - val_accuracy: 0.7143
Epoch 169/1000
178/178 [==============================] - 0s 104us/step - loss: 0.5006 - accuracy: 0.7753 - val_loss: 0.6999 - val_accuracy: 0.7273
Epoch 170/1000
178/178 [==============================] - 0s 115us/step - loss: 0.4888 - accuracy: 0.8146 - val_loss: 0.6076 - val_accuracy: 0.7273
Epoch 171/1000
178/178 [==============================] - 0s 82us/step - loss: 0.4660 - accuracy: 0.8090 - val_loss: 0.6870 - val_accuracy: 0.7273
Epoch 172/1000
178/178 [==============================] - 0s 91us/step - loss: 0.5106 - accuracy: 0.7978 - val_loss: 0.6394 - val_accuracy: 0.7143
Epoch 173/1000
178/178 [==============================] - 0s 83us/step - loss: 0.4650 - accuracy: 0.8258 - val_loss: 0.6402 - val_accuracy: 0.7143
Epoch 174/1000
178/178 [==============================] - 0s 85us/step - loss: 0.4870 - accuracy: 0.7978 - val_loss: 0.6502 - val_a

Epoch 224/1000
178/178 [==============================] - 0s 84us/step - loss: 0.4489 - accuracy: 0.8202 - val_loss: 0.5935 - val_accuracy: 0.7403
Epoch 225/1000
178/178 [==============================] - 0s 94us/step - loss: 0.4831 - accuracy: 0.8034 - val_loss: 0.6021 - val_accuracy: 0.7403
Epoch 226/1000
178/178 [==============================] - 0s 88us/step - loss: 0.4499 - accuracy: 0.8146 - val_loss: 0.5970 - val_accuracy: 0.7273
Epoch 227/1000
178/178 [==============================] - 0s 87us/step - loss: 0.4381 - accuracy: 0.8258 - val_loss: 0.6206 - val_accuracy: 0.7662
Epoch 228/1000
178/178 [==============================] - 0s 94us/step - loss: 0.4527 - accuracy: 0.8202 - val_loss: 0.6282 - val_accuracy: 0.7532
Epoch 229/1000
178/178 [==============================] - 0s 89us/step - loss: 0.4611 - accuracy: 0.8090 - val_loss: 0.5944 - val_accuracy: 0.7403
Epoch 230/1000
178/178 [==============================] - 0s 89us/step - loss: 0.4549 - accuracy: 0.8090 - val_loss: 0

...

Epoch 778/1000
178/178 [==============================] - 0s 121us/step - loss: 0.4075 - accuracy: 0.8202 - val_loss: 0.6412 - val_accuracy: 0.7532
Epoch 779/1000
178/178 [==============================] - 0s 89us/step - loss: 0.4146 - accuracy: 0.8258 - val_loss: 0.6074 - val_accuracy: 0.7662
Epoch 780/1000
178/178 [==============================] - 0s 113us/step - loss: 0.4058 - accuracy: 0.8202 - val_loss: 0.6070 - val_accuracy: 0.7532
Epoch 781/1000
178/178 [==============================] - 0s 104us/step - loss: 0.4111 - accuracy: 0.8202 - val_loss: 0.6168 - val_accuracy: 0.7662
Epoch 782/1000
178/178 [==============================] - 0s 78us/step - loss: 0.4089 - accuracy: 0.8258 - val_loss: 0.6107 - val_accuracy: 0.7662
Epoch 783/1000
178/178 [==============================] - 0s 114us/step - loss: 0.4148 - accuracy: 0.8202 - val_loss: 0.6555 - val_accuracy: 0.7532
Epoch 784/1000
178/178 [==============================] - 0s 92us/step - loss: 0.4243 - accuracy: 0.8258 - val_los

178/178 [==============================] - 0s 117us/step - loss: 0.4058 - accuracy: 0.8258 - val_loss: 0.6283 - val_accuracy: 0.7532
Epoch 834/1000
178/178 [==============================] - 0s 89us/step - loss: 0.4068 - accuracy: 0.8315 - val_loss: 0.6383 - val_accuracy: 0.7532
Epoch 835/1000
178/178 [==============================] - 0s 78us/step - loss: 0.4113 - accuracy: 0.8315 - val_loss: 0.6088 - val_accuracy: 0.7532
Epoch 836/1000
178/178 [==============================] - 0s 94us/step - loss: 0.4108 - accuracy: 0.8258 - val_loss: 0.6224 - val_accuracy: 0.7532
Epoch 837/1000
178/178 [==============================] - 0s 80us/step - loss: 0.4166 - accuracy: 0.8258 - val_loss: 0.6427 - val_accuracy: 0.7532
Epoch 838/1000
178/178 [==============================] - 0s 103us/step - loss: 0.4181 - accuracy: 0.8258 - val_loss: 0.6344 - val_accuracy: 0.7532
Epoch 839/1000
178/178 [==============================] - 0s 90us/step - loss: 0.4304 - accuracy: 0.8202 - val_loss: 0.6099 - val_a

Epoch 889/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4260 - accuracy: 0.8146 - val_loss: 0.6069 - val_accuracy: 0.7662
Epoch 890/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4099 - accuracy: 0.8258 - val_loss: 0.7178 - val_accuracy: 0.7662
Epoch 891/1000
178/178 [==============================] - 0s 85us/step - loss: 0.5410 - accuracy: 0.8034 - val_loss: 0.6237 - val_accuracy: 0.7532
Epoch 892/1000
178/178 [==============================] - 0s 83us/step - loss: 0.4677 - accuracy: 0.8090 - val_loss: 0.6049 - val_accuracy: 0.7662
Epoch 893/1000
178/178 [==============================] - 0s 86us/step - loss: 0.4008 - accuracy: 0.8146 - val_loss: 0.6264 - val_accuracy: 0.7662
Epoch 894/1000
178/178 [==============================] - 0s 85us/step - loss: 0.4042 - accuracy: 0.8258 - val_loss: 0.6207 - val_accuracy: 0.7532
Epoch 895/1000
178/178 [==============================] - 0s 80us/step - loss: 0.3936 - accuracy: 0.8315 - val_loss: 0

Epoch 945/1000
178/178 [==============================] - 0s 103us/step - loss: 0.3954 - accuracy: 0.8202 - val_loss: 0.6127 - val_accuracy: 0.7532
Epoch 946/1000
178/178 [==============================] - 0s 92us/step - loss: 0.3977 - accuracy: 0.8258 - val_loss: 0.6445 - val_accuracy: 0.7532
Epoch 947/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4357 - accuracy: 0.8146 - val_loss: 0.6140 - val_accuracy: 0.7532
Epoch 948/1000
178/178 [==============================] - 0s 88us/step - loss: 0.3933 - accuracy: 0.8315 - val_loss: 0.6093 - val_accuracy: 0.7662
Epoch 949/1000
178/178 [==============================] - 0s 85us/step - loss: 0.4237 - accuracy: 0.8146 - val_loss: 0.6129 - val_accuracy: 0.7662
Epoch 950/1000
178/178 [==============================] - 0s 101us/step - loss: 0.4042 - accuracy: 0.8258 - val_loss: 0.6296 - val_accuracy: 0.7532
Epoch 951/1000
178/178 [==============================] - 0s 98us/step - loss: 0.4035 - accuracy: 0.8146 - val_loss:

178/178 [==============================] - 0s 99us/step - loss: 0.4097 - accuracy: 0.8258 - val_loss: 0.6157 - val_accuracy: 0.7662


In [71]:
acc_test = model2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test*100))



77/77 [==============================] - 0s 89us/step
test accuracy: 76.62%


In [72]:
#### improve neural network
model2_2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [73]:
model2_2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [74]:
model2_2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 629us/step - loss: 60.8036 - accuracy: 0.4382 - val_loss: 0.6925 - val_accuracy: 0.5974
Epoch 2/1000
178/178 [==============================] - 0s 84us/step - loss: 0.7016 - accuracy: 0.5618 - val_loss: 0.6499 - val_accuracy: 0.6623
Epoch 3/1000
178/178 [==============================] - 0s 89us/step - loss: 0.6818 - accuracy: 0.6685 - val_loss: 0.6717 - val_accuracy: 0.7143
Epoch 4/1000
178/178 [==============================] - 0s 106us/step - loss: 0.6652 - accuracy: 0.6854 - val_loss: 0.6472 - val_accuracy: 0.7403
Epoch 5/1000
178/178 [==============================] - 0s 108us/step - loss: 0.6536 - accuracy: 0.7416 - val_loss: 0.6492 - val_accuracy: 0.7403
Epoch 6/1000
178/178 [==============================] - 0s 105us/step - loss: 0.6484 - accuracy: 0.7416 - val_loss: 0.6456 - val_accuracy: 0.7403
Epoch 7/1000
178/178 [==============================] - 0s 91us/step - loss: 0.6

Epoch 57/1000
178/178 [==============================] - 0s 112us/step - loss: 0.5138 - accuracy: 0.8146 - val_loss: 0.6247 - val_accuracy: 0.7403
Epoch 58/1000
178/178 [==============================] - 0s 94us/step - loss: 0.5600 - accuracy: 0.7753 - val_loss: 0.6283 - val_accuracy: 0.7403
Epoch 59/1000
178/178 [==============================] - 0s 91us/step - loss: 0.5611 - accuracy: 0.7921 - val_loss: 0.6166 - val_accuracy: 0.7662
Epoch 60/1000
178/178 [==============================] - 0s 102us/step - loss: 0.6220 - accuracy: 0.7416 - val_loss: 0.6371 - val_accuracy: 0.7143
Epoch 61/1000
178/178 [==============================] - 0s 101us/step - loss: 0.5431 - accuracy: 0.7978 - val_loss: 0.6098 - val_accuracy: 0.7403
Epoch 62/1000
178/178 [==============================] - 0s 113us/step - loss: 0.5261 - accuracy: 0.8146 - val_loss: 0.6087 - val_accuracy: 0.7403
Epoch 63/1000
178/178 [==============================] - 0s 98us/step - loss: 0.5469 - accuracy: 0.7978 - val_loss: 0.61

Epoch 113/1000
178/178 [==============================] - 0s 86us/step - loss: 0.4584 - accuracy: 0.8034 - val_loss: 0.5866 - val_accuracy: 0.7273
Epoch 114/1000
178/178 [==============================] - 0s 107us/step - loss: 0.4536 - accuracy: 0.8090 - val_loss: 0.5866 - val_accuracy: 0.7273
Epoch 115/1000
178/178 [==============================] - 0s 97us/step - loss: 0.5087 - accuracy: 0.7753 - val_loss: 0.5854 - val_accuracy: 0.7273
Epoch 116/1000
178/178 [==============================] - 0s 85us/step - loss: 0.4629 - accuracy: 0.8034 - val_loss: 0.5871 - val_accuracy: 0.7273
Epoch 117/1000
178/178 [==============================] - 0s 91us/step - loss: 0.4428 - accuracy: 0.8146 - val_loss: 0.5884 - val_accuracy: 0.7403
Epoch 118/1000
178/178 [==============================] - 0s 95us/step - loss: 0.4455 - accuracy: 0.8090 - val_loss: 0.7086 - val_accuracy: 0.6883
Epoch 119/1000
178/178 [==============================] - 0s 93us/step - loss: 0.5271 - accuracy: 0.7921 - val_loss: 

178/178 [==============================] - 0s 128us/step - loss: 0.4299 - accuracy: 0.8090 - val_loss: 0.6088 - val_accuracy: 0.7403
Epoch 169/1000
178/178 [==============================] - 0s 126us/step - loss: 0.4493 - accuracy: 0.8034 - val_loss: 0.6128 - val_accuracy: 0.7403
Epoch 170/1000
178/178 [==============================] - 0s 135us/step - loss: 0.4197 - accuracy: 0.8258 - val_loss: 0.6331 - val_accuracy: 0.7273
Epoch 171/1000
178/178 [==============================] - 0s 130us/step - loss: 0.4506 - accuracy: 0.8146 - val_loss: 0.6014 - val_accuracy: 0.7403
Epoch 172/1000
178/178 [==============================] - 0s 117us/step - loss: 0.4297 - accuracy: 0.8090 - val_loss: 0.6477 - val_accuracy: 0.7403
Epoch 173/1000
178/178 [==============================] - 0s 107us/step - loss: 0.4509 - accuracy: 0.8202 - val_loss: 0.6157 - val_accuracy: 0.7403
Epoch 174/1000
178/178 [==============================] - 0s 101us/step - loss: 0.4350 - accuracy: 0.8202 - val_loss: 0.6110 - 

178/178 [==============================] - 0s 89us/step - loss: 0.4100 - accuracy: 0.8315 - val_loss: 0.6244 - val_accuracy: 0.7403
Epoch 224/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4161 - accuracy: 0.8090 - val_loss: 0.6046 - val_accuracy: 0.7403
Epoch 225/1000
178/178 [==============================] - 0s 85us/step - loss: 0.4221 - accuracy: 0.8090 - val_loss: 0.6119 - val_accuracy: 0.7403
Epoch 226/1000
178/178 [==============================] - 0s 96us/step - loss: 0.4088 - accuracy: 0.8258 - val_loss: 0.5985 - val_accuracy: 0.7273
Epoch 227/1000
178/178 [==============================] - 0s 88us/step - loss: 0.4208 - accuracy: 0.8146 - val_loss: 0.5964 - val_accuracy: 0.7273
Epoch 228/1000
178/178 [==============================] - 0s 94us/step - loss: 0.4530 - accuracy: 0.8146 - val_loss: 0.5959 - val_accuracy: 0.7403
Epoch 229/1000
178/178 [==============================] - 0s 92us/step - loss: 0.4046 - accuracy: 0.8258 - val_loss: 0.6335 - val_acc

...

178/178 [==============================] - 0s 85us/step - loss: 0.3705 - accuracy: 0.8315 - val_loss: 0.6376 - val_accuracy: 0.7792
Epoch 727/1000
178/178 [==============================] - 0s 98us/step - loss: 0.3689 - accuracy: 0.8258 - val_loss: 0.6359 - val_accuracy: 0.7792
Epoch 728/1000
178/178 [==============================] - 0s 98us/step - loss: 0.3783 - accuracy: 0.8258 - val_loss: 0.6193 - val_accuracy: 0.7922
Epoch 729/1000
178/178 [==============================] - 0s 93us/step - loss: 0.3724 - accuracy: 0.8258 - val_loss: 0.6342 - val_accuracy: 0.7792
Epoch 730/1000
178/178 [==============================] - 0s 100us/step - loss: 0.3860 - accuracy: 0.8258 - val_loss: 0.5972 - val_accuracy: 0.7792
Epoch 731/1000
178/178 [==============================] - 0s 91us/step - loss: 0.3767 - accuracy: 0.8258 - val_loss: 0.6128 - val_accuracy: 0.7662
Epoch 732/1000
178/178 [==============================] - 0s 101us/step - loss: 0.3674 - accuracy: 0.8258 - val_loss: 0.6376 - val_a

178/178 [==============================] - 0s 105us/step - loss: 0.3667 - accuracy: 0.8258 - val_loss: 0.6137 - val_accuracy: 0.7792
Epoch 782/1000
178/178 [==============================] - 0s 99us/step - loss: 0.3674 - accuracy: 0.8202 - val_loss: 0.6319 - val_accuracy: 0.7922
Epoch 783/1000
178/178 [==============================] - 0s 96us/step - loss: 0.3684 - accuracy: 0.8202 - val_loss: 0.6318 - val_accuracy: 0.7792
Epoch 784/1000
178/178 [==============================] - 0s 100us/step - loss: 0.3870 - accuracy: 0.8202 - val_loss: 0.6212 - val_accuracy: 0.7792
Epoch 785/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3699 - accuracy: 0.8315 - val_loss: 0.6625 - val_accuracy: 0.7662
Epoch 786/1000
178/178 [==============================] - 0s 82us/step - loss: 0.3691 - accuracy: 0.8258 - val_loss: 0.6285 - val_accuracy: 0.7922
Epoch 787/1000
178/178 [==============================] - 0s 100us/step - loss: 0.3653 - accuracy: 0.8315 - val_loss: 0.6277 - val_

Epoch 837/1000
178/178 [==============================] - 0s 93us/step - loss: 0.3947 - accuracy: 0.8315 - val_loss: 0.6174 - val_accuracy: 0.7922
Epoch 838/1000
178/178 [==============================] - 0s 90us/step - loss: 0.3631 - accuracy: 0.8202 - val_loss: 0.6284 - val_accuracy: 0.7792
Epoch 839/1000
178/178 [==============================] - 0s 90us/step - loss: 0.3616 - accuracy: 0.8371 - val_loss: 0.6500 - val_accuracy: 0.7792
Epoch 840/1000
178/178 [==============================] - 0s 97us/step - loss: 0.3609 - accuracy: 0.8427 - val_loss: 0.6591 - val_accuracy: 0.7662
Epoch 841/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3662 - accuracy: 0.8258 - val_loss: 0.6185 - val_accuracy: 0.7792
Epoch 842/1000
178/178 [==============================] - 0s 90us/step - loss: 0.3614 - accuracy: 0.8315 - val_loss: 0.6500 - val_accuracy: 0.7922
Epoch 843/1000
178/178 [==============================] - 0s 90us/step - loss: 0.3611 - accuracy: 0.8427 - val_loss: 0

Epoch 893/1000
178/178 [==============================] - 0s 96us/step - loss: 0.3582 - accuracy: 0.8371 - val_loss: 0.6306 - val_accuracy: 0.7792
Epoch 894/1000
178/178 [==============================] - 0s 90us/step - loss: 0.3576 - accuracy: 0.8315 - val_loss: 0.6406 - val_accuracy: 0.7792
Epoch 895/1000
178/178 [==============================] - 0s 93us/step - loss: 0.3657 - accuracy: 0.8371 - val_loss: 0.6287 - val_accuracy: 0.7532
Epoch 896/1000
178/178 [==============================] - 0s 86us/step - loss: 0.3608 - accuracy: 0.8371 - val_loss: 0.6575 - val_accuracy: 0.7922
Epoch 897/1000
178/178 [==============================] - 0s 88us/step - loss: 0.3509 - accuracy: 0.8371 - val_loss: 0.6392 - val_accuracy: 0.7792
Epoch 898/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3590 - accuracy: 0.8371 - val_loss: 0.6519 - val_accuracy: 0.7792
Epoch 899/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4003 - accuracy: 0.8090 - val_loss: 0

178/178 [==============================] - 0s 87us/step - loss: 0.3619 - accuracy: 0.8483 - val_loss: 0.6448 - val_accuracy: 0.7922
Epoch 949/1000
178/178 [==============================] - 0s 112us/step - loss: 0.3570 - accuracy: 0.8371 - val_loss: 0.6597 - val_accuracy: 0.7922
Epoch 950/1000
178/178 [==============================] - 0s 111us/step - loss: 0.3548 - accuracy: 0.8258 - val_loss: 0.6113 - val_accuracy: 0.7792
Epoch 951/1000
178/178 [==============================] - 0s 91us/step - loss: 0.3781 - accuracy: 0.8202 - val_loss: 0.6482 - val_accuracy: 0.7792
Epoch 952/1000
178/178 [==============================] - 0s 112us/step - loss: 0.3527 - accuracy: 0.8315 - val_loss: 0.6652 - val_accuracy: 0.7792
Epoch 953/1000
178/178 [==============================] - 0s 90us/step - loss: 0.3480 - accuracy: 0.8427 - val_loss: 0.6747 - val_accuracy: 0.7792
Epoch 954/1000
178/178 [==============================] - 0s 98us/step - loss: 0.3607 - accuracy: 0.8427 - val_loss: 0.6384 - val_

In [75]:
acc_test2 = model2_2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 72us/step
test accuracy: 79.22%


In [76]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [77]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [78]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 136 samples, validate on 59 samples
Epoch 1/1000
136/136 [==============================] - 0s 749us/step - loss: 1.4865 - accuracy: 0.4926 - val_loss: 2.3293 - val_accuracy: 0.4576
Epoch 2/1000
136/136 [==============================] - 0s 87us/step - loss: 1.9887 - accuracy: 0.4265 - val_loss: 1.7502 - val_accuracy: 0.4576
Epoch 3/1000
136/136 [==============================] - 0s 98us/step - loss: 2.0188 - accuracy: 0.5000 - val_loss: 1.2969 - val_accuracy: 0.4746
Epoch 4/1000
136/136 [==============================] - 0s 105us/step - loss: 0.7313 - accuracy: 0.6176 - val_loss: 1.2028 - val_accuracy: 0.5424
Epoch 5/1000
136/136 [==============================] - 0s 105us/step - loss: 0.8508 - accuracy: 0.6471 - val_loss: 0.9462 - val_accuracy: 0.6271
Epoch 6/1000
136/136 [==============================] - 0s 109us/step - loss: 0.8986 - accuracy: 0.6471 - val_loss: 6.7069 - val_accuracy: 0.4915
Epoch 7/1000
136/136 [==============================] - 0s 121us/step - loss: 2.0

136/136 [==============================] - 0s 99us/step - loss: 0.2987 - accuracy: 0.9338 - val_loss: 0.7263 - val_accuracy: 0.7458
Epoch 57/1000
136/136 [==============================] - 0s 114us/step - loss: 0.3586 - accuracy: 0.8897 - val_loss: 0.4397 - val_accuracy: 0.7627
Epoch 58/1000
136/136 [==============================] - 0s 104us/step - loss: 0.3089 - accuracy: 0.8971 - val_loss: 0.4335 - val_accuracy: 0.7458
Epoch 59/1000
136/136 [==============================] - 0s 107us/step - loss: 0.3076 - accuracy: 0.9265 - val_loss: 0.4135 - val_accuracy: 0.9153
Epoch 60/1000
136/136 [==============================] - 0s 109us/step - loss: 0.3058 - accuracy: 0.9412 - val_loss: 0.4937 - val_accuracy: 0.7458
Epoch 61/1000
136/136 [==============================] - 0s 111us/step - loss: 0.3204 - accuracy: 0.9044 - val_loss: 0.4192 - val_accuracy: 0.8136
Epoch 62/1000
136/136 [==============================] - 0s 103us/step - loss: 0.2998 - accuracy: 0.8897 - val_loss: 0.4287 - val_acc

Epoch 112/1000
136/136 [==============================] - 0s 114us/step - loss: 0.2571 - accuracy: 0.8750 - val_loss: 0.4142 - val_accuracy: 0.8136
Epoch 113/1000
136/136 [==============================] - 0s 111us/step - loss: 0.2677 - accuracy: 0.8750 - val_loss: 0.4047 - val_accuracy: 0.8136
Epoch 114/1000
136/136 [==============================] - 0s 113us/step - loss: 0.2506 - accuracy: 0.8750 - val_loss: 0.3964 - val_accuracy: 0.8305
Epoch 115/1000
136/136 [==============================] - 0s 110us/step - loss: 0.2552 - accuracy: 0.8750 - val_loss: 0.3993 - val_accuracy: 0.7627
Epoch 116/1000
136/136 [==============================] - 0s 108us/step - loss: 0.2465 - accuracy: 0.8676 - val_loss: 0.3996 - val_accuracy: 0.7458
Epoch 117/1000
136/136 [==============================] - 0s 102us/step - loss: 0.2481 - accuracy: 0.8603 - val_loss: 0.4016 - val_accuracy: 0.6610
Epoch 118/1000
136/136 [==============================] - 0s 110us/step - loss: 0.2495 - accuracy: 0.8309 - val_

136/136 [==============================] - 0s 111us/step - loss: 0.2272 - accuracy: 0.9118 - val_loss: 0.5332 - val_accuracy: 0.6610
Epoch 168/1000
136/136 [==============================] - 0s 109us/step - loss: 0.2643 - accuracy: 0.8897 - val_loss: 0.4330 - val_accuracy: 0.6949
Epoch 169/1000
136/136 [==============================] - 0s 105us/step - loss: 0.2357 - accuracy: 0.8971 - val_loss: 0.5178 - val_accuracy: 0.8305
Epoch 170/1000
136/136 [==============================] - 0s 127us/step - loss: 0.2957 - accuracy: 0.8897 - val_loss: 0.5414 - val_accuracy: 0.7458
Epoch 171/1000
136/136 [==============================] - 0s 100us/step - loss: 0.2424 - accuracy: 0.8971 - val_loss: 0.4041 - val_accuracy: 0.8644
Epoch 172/1000
136/136 [==============================] - 0s 109us/step - loss: 0.2314 - accuracy: 0.9338 - val_loss: 0.4122 - val_accuracy: 0.7288
Epoch 173/1000
136/136 [==============================] - 0s 112us/step - loss: 0.2253 - accuracy: 0.8897 - val_loss: 0.3861 - 

136/136 [==============================] - 0s 100us/step - loss: 0.2115 - accuracy: 0.8676 - val_loss: 0.3826 - val_accuracy: 0.8136
Epoch 223/1000
136/136 [==============================] - 0s 99us/step - loss: 0.2132 - accuracy: 0.8824 - val_loss: 0.5185 - val_accuracy: 0.7797
Epoch 224/1000
136/136 [==============================] - 0s 107us/step - loss: 0.2954 - accuracy: 0.8235 - val_loss: 0.3807 - val_accuracy: 0.8136
Epoch 225/1000
136/136 [==============================] - 0s 104us/step - loss: 0.2080 - accuracy: 0.8750 - val_loss: 0.3919 - val_accuracy: 0.8136
Epoch 226/1000
136/136 [==============================] - 0s 101us/step - loss: 0.2145 - accuracy: 0.8750 - val_loss: 0.3872 - val_accuracy: 0.8136
Epoch 227/1000
136/136 [==============================] - 0s 101us/step - loss: 0.2110 - accuracy: 0.8824 - val_loss: 0.3814 - val_accuracy: 0.8136
Epoch 228/1000
136/136 [==============================] - 0s 111us/step - loss: 0.2068 - accuracy: 0.8971 - val_loss: 0.3821 - v

...

136/136 [==============================] - 0s 84us/step - loss: 0.2375 - accuracy: 0.8676 - val_loss: 0.5642 - val_accuracy: 0.6949
Epoch 773/1000
136/136 [==============================] - 0s 86us/step - loss: 0.2010 - accuracy: 0.8971 - val_loss: 0.4644 - val_accuracy: 0.7119
Epoch 774/1000
136/136 [==============================] - 0s 101us/step - loss: 0.1836 - accuracy: 0.8971 - val_loss: 0.4203 - val_accuracy: 0.7458
Epoch 775/1000
136/136 [==============================] - 0s 114us/step - loss: 0.1674 - accuracy: 0.8971 - val_loss: 0.4203 - val_accuracy: 0.7458
Epoch 776/1000
136/136 [==============================] - 0s 113us/step - loss: 0.1732 - accuracy: 0.9044 - val_loss: 0.4644 - val_accuracy: 0.7119
Epoch 777/1000
136/136 [==============================] - 0s 117us/step - loss: 0.1797 - accuracy: 0.8971 - val_loss: 0.4178 - val_accuracy: 0.7627
Epoch 778/1000
136/136 [==============================] - 0s 120us/step - loss: 0.1650 - accuracy: 0.9118 - val_loss: 0.4302 - va

136/136 [==============================] - 0s 135us/step - loss: 0.1682 - accuracy: 0.9044 - val_loss: 0.4251 - val_accuracy: 0.7288
Epoch 828/1000
136/136 [==============================] - 0s 132us/step - loss: 0.1673 - accuracy: 0.9118 - val_loss: 0.4421 - val_accuracy: 0.7119
Epoch 829/1000
136/136 [==============================] - 0s 146us/step - loss: 0.1864 - accuracy: 0.8897 - val_loss: 0.4338 - val_accuracy: 0.7119
Epoch 830/1000
136/136 [==============================] - 0s 116us/step - loss: 0.1693 - accuracy: 0.8971 - val_loss: 0.4205 - val_accuracy: 0.7288
Epoch 831/1000
136/136 [==============================] - 0s 148us/step - loss: 0.1745 - accuracy: 0.8971 - val_loss: 0.4299 - val_accuracy: 0.7119
Epoch 832/1000
136/136 [==============================] - 0s 99us/step - loss: 0.1729 - accuracy: 0.8971 - val_loss: 0.4213 - val_accuracy: 0.7288
Epoch 833/1000
136/136 [==============================] - 0s 126us/step - loss: 0.1654 - accuracy: 0.9044 - val_loss: 0.4242 - v

136/136 [==============================] - 0s 108us/step - loss: 0.1650 - accuracy: 0.9118 - val_loss: 0.4293 - val_accuracy: 0.7119
Epoch 883/1000
136/136 [==============================] - 0s 107us/step - loss: 0.1635 - accuracy: 0.8971 - val_loss: 0.5427 - val_accuracy: 0.7119
Epoch 884/1000
136/136 [==============================] - 0s 112us/step - loss: 0.3896 - accuracy: 0.8529 - val_loss: 0.4880 - val_accuracy: 0.6949
Epoch 885/1000
136/136 [==============================] - 0s 105us/step - loss: 0.2001 - accuracy: 0.8897 - val_loss: 0.4148 - val_accuracy: 0.7458
Epoch 886/1000
136/136 [==============================] - 0s 98us/step - loss: 0.1634 - accuracy: 0.9044 - val_loss: 0.4150 - val_accuracy: 0.7458
Epoch 887/1000
136/136 [==============================] - 0s 103us/step - loss: 0.1714 - accuracy: 0.8971 - val_loss: 0.5169 - val_accuracy: 0.7119
Epoch 888/1000
136/136 [==============================] - 0s 100us/step - loss: 0.2057 - accuracy: 0.8971 - val_loss: 0.4147 - v

136/136 [==============================] - 0s 93us/step - loss: 0.1625 - accuracy: 0.9044 - val_loss: 0.4258 - val_accuracy: 0.7288
Epoch 938/1000
136/136 [==============================] - 0s 90us/step - loss: 0.1685 - accuracy: 0.9044 - val_loss: 0.4697 - val_accuracy: 0.7119
Epoch 939/1000
136/136 [==============================] - 0s 103us/step - loss: 0.1722 - accuracy: 0.8971 - val_loss: 0.4233 - val_accuracy: 0.7458
Epoch 940/1000
136/136 [==============================] - 0s 100us/step - loss: 0.1648 - accuracy: 0.9044 - val_loss: 0.4405 - val_accuracy: 0.7119
Epoch 941/1000
136/136 [==============================] - 0s 89us/step - loss: 0.1651 - accuracy: 0.8971 - val_loss: 0.4231 - val_accuracy: 0.7458
Epoch 942/1000
136/136 [==============================] - 0s 92us/step - loss: 0.1639 - accuracy: 0.9044 - val_loss: 0.4295 - val_accuracy: 0.7288
Epoch 943/1000
136/136 [==============================] - 0s 110us/step - loss: 0.1621 - accuracy: 0.9118 - val_loss: 0.4227 - val_

136/136 [==============================] - 0s 108us/step - loss: 0.1611 - accuracy: 0.9044 - val_loss: 0.4314 - val_accuracy: 0.7288
Epoch 993/1000
136/136 [==============================] - 0s 119us/step - loss: 0.1666 - accuracy: 0.9044 - val_loss: 0.4265 - val_accuracy: 0.7627
Epoch 994/1000
136/136 [==============================] - 0s 114us/step - loss: 0.1640 - accuracy: 0.9044 - val_loss: 0.4284 - val_accuracy: 0.7458
Epoch 995/1000
136/136 [==============================] - 0s 91us/step - loss: 0.1633 - accuracy: 0.9044 - val_loss: 0.4255 - val_accuracy: 0.7627
Epoch 996/1000
136/136 [==============================] - 0s 111us/step - loss: 0.1622 - accuracy: 0.9044 - val_loss: 0.4259 - val_accuracy: 0.7627
Epoch 997/1000
136/136 [==============================] - 0s 112us/step - loss: 0.1581 - accuracy: 0.9118 - val_loss: 0.4321 - val_accuracy: 0.7288
Epoch 998/1000
136/136 [==============================] - 0s 89us/step - loss: 0.1620 - accuracy: 0.9044 - val_loss: 0.4324 - va

In [79]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


59/59 [==============================] - 0s 84us/step
combination test accuracy: 74.58%


In [119]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [120]:
model2_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [121]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 136 samples, validate on 59 samples
Epoch 1/1000
136/136 [==============================] - 0s 1ms/step - loss: 6.4186 - accuracy: 0.5441 - val_loss: 5.4599 - val_accuracy: 0.5424
Epoch 2/1000
136/136 [==============================] - 0s 189us/step - loss: 4.1918 - accuracy: 0.5441 - val_loss: 1.3073 - val_accuracy: 0.4746
Epoch 3/1000
136/136 [==============================] - 0s 188us/step - loss: 2.8187 - accuracy: 0.5809 - val_loss: 0.8632 - val_accuracy: 0.4746
Epoch 4/1000
136/136 [==============================] - 0s 201us/step - loss: 2.3546 - accuracy: 0.6765 - val_loss: 0.7576 - val_accuracy: 0.5424
Epoch 5/1000
136/136 [==============================] - 0s 212us/step - loss: 1.6735 - accuracy: 0.6912 - val_loss: 0.7033 - val_accuracy: 0.5424
Epoch 6/1000
136/136 [==============================] - 0s 223us/step - loss: 1.5648 - accuracy: 0.7206 - val_loss: 0.6965 - val_accuracy: 0.5424
Epoch 7/1000
136/136 [==============================] - 0s 214us/step - loss: 2.5

136/136 [==============================] - 0s 181us/step - loss: 1.9056 - accuracy: 0.8015 - val_loss: 0.5697 - val_accuracy: 0.7458
Epoch 57/1000
136/136 [==============================] - 0s 204us/step - loss: 1.5674 - accuracy: 0.8162 - val_loss: 0.5698 - val_accuracy: 0.7627
Epoch 58/1000
136/136 [==============================] - 0s 176us/step - loss: 1.9924 - accuracy: 0.8088 - val_loss: 0.5678 - val_accuracy: 0.7458
Epoch 59/1000
136/136 [==============================] - 0s 171us/step - loss: 1.6462 - accuracy: 0.8456 - val_loss: 0.5652 - val_accuracy: 0.7797
Epoch 60/1000
136/136 [==============================] - 0s 185us/step - loss: 1.5560 - accuracy: 0.8235 - val_loss: 0.5612 - val_accuracy: 0.7797
Epoch 61/1000
136/136 [==============================] - 0s 177us/step - loss: 1.8782 - accuracy: 0.8088 - val_loss: 0.5624 - val_accuracy: 0.7797
Epoch 62/1000
136/136 [==============================] - 0s 173us/step - loss: 2.4019 - accuracy: 0.8015 - val_loss: 0.5600 - val_ac

136/136 [==============================] - 0s 232us/step - loss: 2.6254 - accuracy: 0.7426 - val_loss: 0.4730 - val_accuracy: 0.7458
Epoch 112/1000
136/136 [==============================] - 0s 143us/step - loss: 1.3505 - accuracy: 0.8603 - val_loss: 0.4793 - val_accuracy: 0.7458
Epoch 113/1000
136/136 [==============================] - 0s 170us/step - loss: 1.8431 - accuracy: 0.8015 - val_loss: 0.5193 - val_accuracy: 0.7966
Epoch 114/1000
136/136 [==============================] - 0s 177us/step - loss: 1.6029 - accuracy: 0.8235 - val_loss: 0.5057 - val_accuracy: 0.7288
Epoch 115/1000
136/136 [==============================] - 0s 175us/step - loss: 1.4961 - accuracy: 0.8235 - val_loss: 0.5066 - val_accuracy: 0.7288
Epoch 116/1000
136/136 [==============================] - 0s 159us/step - loss: 1.6104 - accuracy: 0.8088 - val_loss: 0.4999 - val_accuracy: 0.7288
Epoch 117/1000
136/136 [==============================] - 0s 167us/step - loss: 2.2520 - accuracy: 0.7941 - val_loss: 0.5027 - 

136/136 [==============================] - 0s 165us/step - loss: 2.0266 - accuracy: 0.7794 - val_loss: 0.4718 - val_accuracy: 0.8305
Epoch 167/1000
136/136 [==============================] - 0s 170us/step - loss: 2.0226 - accuracy: 0.7941 - val_loss: 0.4452 - val_accuracy: 0.7966
Epoch 168/1000
136/136 [==============================] - 0s 190us/step - loss: 2.0305 - accuracy: 0.7941 - val_loss: 0.4539 - val_accuracy: 0.8136
Epoch 169/1000
136/136 [==============================] - 0s 176us/step - loss: 1.2405 - accuracy: 0.8529 - val_loss: 0.4989 - val_accuracy: 0.8136
Epoch 170/1000
136/136 [==============================] - 0s 175us/step - loss: 1.2703 - accuracy: 0.8309 - val_loss: 0.4887 - val_accuracy: 0.7966
Epoch 171/1000
136/136 [==============================] - 0s 175us/step - loss: 1.4599 - accuracy: 0.8382 - val_loss: 0.4858 - val_accuracy: 0.7966
Epoch 172/1000
136/136 [==============================] - 0s 193us/step - loss: 1.9999 - accuracy: 0.8088 - val_loss: 0.4929 - 

136/136 [==============================] - 0s 166us/step - loss: 1.7988 - accuracy: 0.8162 - val_loss: 0.4822 - val_accuracy: 0.7458
Epoch 222/1000
136/136 [==============================] - 0s 194us/step - loss: 1.2415 - accuracy: 0.8456 - val_loss: 0.4746 - val_accuracy: 0.7458
Epoch 223/1000
136/136 [==============================] - 0s 181us/step - loss: 1.5419 - accuracy: 0.8529 - val_loss: 0.4805 - val_accuracy: 0.7458
Epoch 224/1000
136/136 [==============================] - 0s 204us/step - loss: 2.0328 - accuracy: 0.7868 - val_loss: 0.4846 - val_accuracy: 0.7288
Epoch 225/1000
136/136 [==============================] - 0s 170us/step - loss: 1.3427 - accuracy: 0.8456 - val_loss: 0.4805 - val_accuracy: 0.7458
Epoch 226/1000
136/136 [==============================] - 0s 153us/step - loss: 2.1205 - accuracy: 0.7941 - val_loss: 0.4708 - val_accuracy: 0.7458
Epoch 227/1000
136/136 [==============================] - 0s 165us/step - loss: 1.1332 - accuracy: 0.8529 - val_loss: 0.4728 - 

...

136/136 [==============================] - 0s 185us/step - loss: 1.3412 - accuracy: 0.8382 - val_loss: 0.5096 - val_accuracy: 0.7458
Epoch 772/1000
136/136 [==============================] - 0s 178us/step - loss: 1.2249 - accuracy: 0.8456 - val_loss: 0.5054 - val_accuracy: 0.7458
Epoch 773/1000
136/136 [==============================] - 0s 171us/step - loss: 1.4427 - accuracy: 0.8382 - val_loss: 0.5064 - val_accuracy: 0.7458
Epoch 774/1000
136/136 [==============================] - 0s 161us/step - loss: 1.5532 - accuracy: 0.8309 - val_loss: 0.5074 - val_accuracy: 0.7458
Epoch 775/1000
136/136 [==============================] - 0s 167us/step - loss: 1.1171 - accuracy: 0.8529 - val_loss: 0.5101 - val_accuracy: 0.7458
Epoch 776/1000
136/136 [==============================] - 0s 158us/step - loss: 1.2457 - accuracy: 0.8309 - val_loss: 0.5104 - val_accuracy: 0.7458
Epoch 777/1000
136/136 [==============================] - 0s 172us/step - loss: 1.4749 - accuracy: 0.8162 - val_loss: 0.5115 - 

136/136 [==============================] - 0s 145us/step - loss: 1.6869 - accuracy: 0.8088 - val_loss: 0.5084 - val_accuracy: 0.7458
Epoch 827/1000
136/136 [==============================] - 0s 150us/step - loss: 1.8875 - accuracy: 0.8088 - val_loss: 0.5092 - val_accuracy: 0.7458
Epoch 828/1000
136/136 [==============================] - 0s 170us/step - loss: 1.5667 - accuracy: 0.8235 - val_loss: 0.5116 - val_accuracy: 0.7458
Epoch 829/1000
136/136 [==============================] - 0s 189us/step - loss: 1.6614 - accuracy: 0.8235 - val_loss: 0.5131 - val_accuracy: 0.7458
Epoch 830/1000
136/136 [==============================] - 0s 189us/step - loss: 1.7928 - accuracy: 0.8015 - val_loss: 0.5135 - val_accuracy: 0.7458
Epoch 831/1000
136/136 [==============================] - 0s 184us/step - loss: 1.2417 - accuracy: 0.8309 - val_loss: 0.5065 - val_accuracy: 0.7458
Epoch 832/1000
136/136 [==============================] - 0s 171us/step - loss: 1.2114 - accuracy: 0.8603 - val_loss: 0.5117 - 

136/136 [==============================] - 0s 171us/step - loss: 1.8951 - accuracy: 0.8088 - val_loss: 0.5179 - val_accuracy: 0.7458
Epoch 882/1000
136/136 [==============================] - 0s 186us/step - loss: 1.5637 - accuracy: 0.8235 - val_loss: 0.5139 - val_accuracy: 0.7458
Epoch 883/1000
136/136 [==============================] - 0s 175us/step - loss: 0.7711 - accuracy: 0.8824 - val_loss: 0.5086 - val_accuracy: 0.7458
Epoch 884/1000
136/136 [==============================] - 0s 179us/step - loss: 0.9899 - accuracy: 0.8676 - val_loss: 0.5102 - val_accuracy: 0.7458
Epoch 885/1000
136/136 [==============================] - 0s 162us/step - loss: 1.5610 - accuracy: 0.8235 - val_loss: 0.5114 - val_accuracy: 0.7458
Epoch 886/1000
136/136 [==============================] - 0s 168us/step - loss: 1.3329 - accuracy: 0.8456 - val_loss: 0.5120 - val_accuracy: 0.7458
Epoch 887/1000
136/136 [==============================] - 0s 158us/step - loss: 1.4573 - accuracy: 0.8235 - val_loss: 0.5150 - 

136/136 [==============================] - 0s 172us/step - loss: 1.0912 - accuracy: 0.8676 - val_loss: 0.5140 - val_accuracy: 0.7458
Epoch 937/1000
136/136 [==============================] - 0s 187us/step - loss: 1.7638 - accuracy: 0.8235 - val_loss: 0.5151 - val_accuracy: 0.7458
Epoch 938/1000
136/136 [==============================] - 0s 174us/step - loss: 1.0672 - accuracy: 0.8897 - val_loss: 0.5141 - val_accuracy: 0.7458
Epoch 939/1000
136/136 [==============================] - 0s 172us/step - loss: 1.7683 - accuracy: 0.8235 - val_loss: 0.5138 - val_accuracy: 0.7458
Epoch 940/1000
136/136 [==============================] - 0s 173us/step - loss: 1.6529 - accuracy: 0.8309 - val_loss: 0.5149 - val_accuracy: 0.7458
Epoch 941/1000
136/136 [==============================] - 0s 170us/step - loss: 0.9782 - accuracy: 0.8750 - val_loss: 0.5100 - val_accuracy: 0.7458
Epoch 942/1000
136/136 [==============================] - 0s 176us/step - loss: 1.4672 - accuracy: 0.8162 - val_loss: 0.5142 - 

136/136 [==============================] - 0s 162us/step - loss: 2.5437 - accuracy: 0.7794 - val_loss: 0.5140 - val_accuracy: 0.7458
Epoch 992/1000
136/136 [==============================] - 0s 209us/step - loss: 1.1115 - accuracy: 0.8529 - val_loss: 0.5142 - val_accuracy: 0.7458
Epoch 993/1000
136/136 [==============================] - 0s 177us/step - loss: 1.7817 - accuracy: 0.8088 - val_loss: 0.5168 - val_accuracy: 0.7458
Epoch 994/1000
136/136 [==============================] - 0s 197us/step - loss: 1.9116 - accuracy: 0.7941 - val_loss: 0.5194 - val_accuracy: 0.7458
Epoch 995/1000
136/136 [==============================] - 0s 165us/step - loss: 1.2223 - accuracy: 0.8456 - val_loss: 0.5114 - val_accuracy: 0.7458
Epoch 996/1000
136/136 [==============================] - 0s 166us/step - loss: 1.4192 - accuracy: 0.8529 - val_loss: 0.5129 - val_accuracy: 0.7458
Epoch 997/1000
136/136 [==============================] - 0s 159us/step - loss: 1.5412 - accuracy: 0.8382 - val_loss: 0.5104 - 

In [122]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

59/59 [==============================] - 0s 102us/step
combination test accuracy: 74.58%


In [84]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [85]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [86]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 454us/step - loss: 22.4757 - accuracy: 0.5325 - val_loss: 0.6728 - val_accuracy: 0.5657
Epoch 2/1000
231/231 [==============================] - 0s 71us/step - loss: 0.6827 - accuracy: 0.5758 - val_loss: 0.6699 - val_accuracy: 0.5657
Epoch 3/1000
231/231 [==============================] - 0s 82us/step - loss: 0.7011 - accuracy: 0.6017 - val_loss: 0.6617 - val_accuracy: 0.6970
Epoch 4/1000
231/231 [==============================] - 0s 86us/step - loss: 0.6792 - accuracy: 0.5974 - val_loss: 0.6554 - val_accuracy: 0.6970
Epoch 5/1000
231/231 [==============================] - 0s 84us/step - loss: 0.6784 - accuracy: 0.6190 - val_loss: 0.6705 - val_accuracy: 0.5455
Epoch 6/1000
231/231 [==============================] - 0s 87us/step - loss: 0.6680 - accuracy: 0.6104 - val_loss: 0.6562 - val_accuracy: 0.6970
Epoch 7/1000
231/231 [==============================] - 0s 89us/step - loss: 0.6795

231/231 [==============================] - 0s 80us/step - loss: 0.5787 - accuracy: 0.7143 - val_loss: 0.5746 - val_accuracy: 0.7677
Epoch 57/1000
231/231 [==============================] - 0s 92us/step - loss: 0.5657 - accuracy: 0.7143 - val_loss: 0.5711 - val_accuracy: 0.7576
Epoch 58/1000
231/231 [==============================] - 0s 78us/step - loss: 0.5637 - accuracy: 0.7186 - val_loss: 0.5783 - val_accuracy: 0.7374
Epoch 59/1000
231/231 [==============================] - 0s 78us/step - loss: 0.6008 - accuracy: 0.6840 - val_loss: 0.5807 - val_accuracy: 0.7677
Epoch 60/1000
231/231 [==============================] - 0s 80us/step - loss: 0.5662 - accuracy: 0.7100 - val_loss: 0.5674 - val_accuracy: 0.7677
Epoch 61/1000
231/231 [==============================] - 0s 81us/step - loss: 0.5547 - accuracy: 0.7229 - val_loss: 0.5745 - val_accuracy: 0.7677
Epoch 62/1000
231/231 [==============================] - 0s 84us/step - loss: 0.5662 - accuracy: 0.7273 - val_loss: 0.6093 - val_accuracy:

231/231 [==============================] - 0s 84us/step - loss: 0.5330 - accuracy: 0.7229 - val_loss: 0.5285 - val_accuracy: 0.7576
Epoch 113/1000
231/231 [==============================] - 0s 83us/step - loss: 0.5265 - accuracy: 0.7273 - val_loss: 0.5278 - val_accuracy: 0.7374
Epoch 114/1000
231/231 [==============================] - 0s 91us/step - loss: 0.5266 - accuracy: 0.7273 - val_loss: 0.5408 - val_accuracy: 0.7374
Epoch 115/1000
231/231 [==============================] - 0s 83us/step - loss: 0.5291 - accuracy: 0.7359 - val_loss: 0.6619 - val_accuracy: 0.6869
Epoch 116/1000
231/231 [==============================] - 0s 92us/step - loss: 0.6128 - accuracy: 0.6970 - val_loss: 0.5388 - val_accuracy: 0.7374
Epoch 117/1000
231/231 [==============================] - 0s 76us/step - loss: 0.5282 - accuracy: 0.7229 - val_loss: 0.5392 - val_accuracy: 0.7374
Epoch 118/1000
231/231 [==============================] - 0s 89us/step - loss: 0.5424 - accuracy: 0.7100 - val_loss: 0.6753 - val_acc

Epoch 168/1000
231/231 [==============================] - 0s 78us/step - loss: 0.5578 - accuracy: 0.7273 - val_loss: 0.5205 - val_accuracy: 0.7475
Epoch 169/1000
231/231 [==============================] - 0s 96us/step - loss: 0.5205 - accuracy: 0.7359 - val_loss: 0.5688 - val_accuracy: 0.7172
Epoch 170/1000
231/231 [==============================] - 0s 86us/step - loss: 0.5338 - accuracy: 0.7100 - val_loss: 0.5413 - val_accuracy: 0.7374
Epoch 171/1000
231/231 [==============================] - 0s 78us/step - loss: 0.5305 - accuracy: 0.7186 - val_loss: 0.5186 - val_accuracy: 0.7677
Epoch 172/1000
231/231 [==============================] - 0s 89us/step - loss: 0.5126 - accuracy: 0.7359 - val_loss: 0.5206 - val_accuracy: 0.7374
Epoch 173/1000
231/231 [==============================] - 0s 90us/step - loss: 0.5132 - accuracy: 0.7316 - val_loss: 0.5178 - val_accuracy: 0.7475
Epoch 174/1000
231/231 [==============================] - 0s 89us/step - loss: 0.5191 - accuracy: 0.7273 - val_loss: 0

Epoch 224/1000
231/231 [==============================] - 0s 90us/step - loss: 0.5103 - accuracy: 0.7316 - val_loss: 0.5102 - val_accuracy: 0.7677
Epoch 225/1000
231/231 [==============================] - 0s 97us/step - loss: 0.5034 - accuracy: 0.7316 - val_loss: 0.5089 - val_accuracy: 0.7677
Epoch 226/1000
231/231 [==============================] - 0s 85us/step - loss: 0.5164 - accuracy: 0.7229 - val_loss: 0.5079 - val_accuracy: 0.7677
Epoch 227/1000
231/231 [==============================] - 0s 87us/step - loss: 0.5143 - accuracy: 0.7273 - val_loss: 0.5089 - val_accuracy: 0.7677
Epoch 228/1000
231/231 [==============================] - 0s 86us/step - loss: 0.5173 - accuracy: 0.7403 - val_loss: 0.5637 - val_accuracy: 0.7475
Epoch 229/1000
231/231 [==============================] - 0s 100us/step - loss: 0.5513 - accuracy: 0.7229 - val_loss: 0.5141 - val_accuracy: 0.7677
Epoch 230/1000
231/231 [==============================] - 0s 80us/step - loss: 0.5153 - accuracy: 0.7359 - val_loss: 

...

Epoch 727/1000
231/231 [==============================] - 0s 86us/step - loss: 0.4690 - accuracy: 0.7446 - val_loss: 0.5639 - val_accuracy: 0.7475
Epoch 728/1000
231/231 [==============================] - 0s 78us/step - loss: 0.5020 - accuracy: 0.7576 - val_loss: 0.5425 - val_accuracy: 0.7273
Epoch 729/1000
231/231 [==============================] - 0s 79us/step - loss: 0.4716 - accuracy: 0.7532 - val_loss: 0.4889 - val_accuracy: 0.7576
Epoch 730/1000
231/231 [==============================] - 0s 79us/step - loss: 0.4567 - accuracy: 0.7576 - val_loss: 0.6753 - val_accuracy: 0.7273
Epoch 731/1000
231/231 [==============================] - 0s 80us/step - loss: 0.5491 - accuracy: 0.7619 - val_loss: 0.4823 - val_accuracy: 0.7677
Epoch 732/1000
231/231 [==============================] - 0s 76us/step - loss: 0.4505 - accuracy: 0.7706 - val_loss: 0.4811 - val_accuracy: 0.7677
Epoch 733/1000
231/231 [==============================] - 0s 76us/step - loss: 0.4550 - accuracy: 0.7619 - val_loss: 0

Epoch 783/1000
231/231 [==============================] - 0s 79us/step - loss: 0.4731 - accuracy: 0.7489 - val_loss: 0.4872 - val_accuracy: 0.7778
Epoch 784/1000
231/231 [==============================] - 0s 81us/step - loss: 0.4485 - accuracy: 0.7749 - val_loss: 0.4939 - val_accuracy: 0.7778
Epoch 785/1000
231/231 [==============================] - 0s 78us/step - loss: 0.4479 - accuracy: 0.7706 - val_loss: 0.4862 - val_accuracy: 0.7677
Epoch 786/1000
231/231 [==============================] - 0s 81us/step - loss: 0.4697 - accuracy: 0.7749 - val_loss: 0.4808 - val_accuracy: 0.7879
Epoch 787/1000
231/231 [==============================] - 0s 78us/step - loss: 0.4417 - accuracy: 0.7749 - val_loss: 0.4798 - val_accuracy: 0.7677
Epoch 788/1000
231/231 [==============================] - 0s 82us/step - loss: 0.4430 - accuracy: 0.7706 - val_loss: 0.4801 - val_accuracy: 0.7677
Epoch 789/1000
231/231 [==============================] - 0s 81us/step - loss: 0.4445 - accuracy: 0.7749 - val_loss: 0

231/231 [==============================] - 0s 141us/step - loss: 0.4355 - accuracy: 0.7662 - val_loss: 0.5042 - val_accuracy: 0.7677
Epoch 839/1000
231/231 [==============================] - 0s 137us/step - loss: 0.4557 - accuracy: 0.7662 - val_loss: 0.4872 - val_accuracy: 0.7879
Epoch 840/1000
231/231 [==============================] - 0s 115us/step - loss: 0.4433 - accuracy: 0.7965 - val_loss: 0.5244 - val_accuracy: 0.7576
Epoch 841/1000
231/231 [==============================] - 0s 109us/step - loss: 0.4644 - accuracy: 0.7662 - val_loss: 0.4916 - val_accuracy: 0.7677
Epoch 842/1000
231/231 [==============================] - 0s 121us/step - loss: 0.4369 - accuracy: 0.7792 - val_loss: 0.4754 - val_accuracy: 0.7677
Epoch 843/1000
231/231 [==============================] - 0s 116us/step - loss: 0.4424 - accuracy: 0.7706 - val_loss: 0.4782 - val_accuracy: 0.7980
Epoch 844/1000
231/231 [==============================] - 0s 107us/step - loss: 0.4467 - accuracy: 0.7792 - val_loss: 0.4755 - 

Epoch 894/1000
231/231 [==============================] - 0s 81us/step - loss: 0.4376 - accuracy: 0.7749 - val_loss: 0.4810 - val_accuracy: 0.7879
Epoch 895/1000
231/231 [==============================] - 0s 79us/step - loss: 0.4436 - accuracy: 0.7792 - val_loss: 0.4955 - val_accuracy: 0.7677
Epoch 896/1000
231/231 [==============================] - 0s 79us/step - loss: 0.4402 - accuracy: 0.7749 - val_loss: 0.5309 - val_accuracy: 0.7172
Epoch 897/1000
231/231 [==============================] - 0s 77us/step - loss: 0.4800 - accuracy: 0.7576 - val_loss: 0.4899 - val_accuracy: 0.7677
Epoch 898/1000
231/231 [==============================] - 0s 87us/step - loss: 0.4772 - accuracy: 0.7662 - val_loss: 0.4796 - val_accuracy: 0.7475
Epoch 899/1000
231/231 [==============================] - 0s 83us/step - loss: 0.4462 - accuracy: 0.7576 - val_loss: 0.4763 - val_accuracy: 0.7778
Epoch 900/1000
231/231 [==============================] - 0s 78us/step - loss: 0.4388 - accuracy: 0.7749 - val_loss: 0

Epoch 950/1000
231/231 [==============================] - 0s 99us/step - loss: 0.4914 - accuracy: 0.7879 - val_loss: 0.4759 - val_accuracy: 0.7879
Epoch 951/1000
231/231 [==============================] - 0s 76us/step - loss: 0.4298 - accuracy: 0.7835 - val_loss: 0.4761 - val_accuracy: 0.7879
Epoch 952/1000
231/231 [==============================] - 0s 83us/step - loss: 0.4389 - accuracy: 0.7706 - val_loss: 0.4920 - val_accuracy: 0.7879
Epoch 953/1000
231/231 [==============================] - 0s 77us/step - loss: 0.4437 - accuracy: 0.7749 - val_loss: 0.4825 - val_accuracy: 0.7778
Epoch 954/1000
231/231 [==============================] - 0s 81us/step - loss: 0.4337 - accuracy: 0.7706 - val_loss: 0.4783 - val_accuracy: 0.7879
Epoch 955/1000
231/231 [==============================] - 0s 79us/step - loss: 0.4475 - accuracy: 0.7749 - val_loss: 0.4865 - val_accuracy: 0.7778
Epoch 956/1000
231/231 [==============================] - 0s 80us/step - loss: 0.4355 - accuracy: 0.7749 - val_loss: 0

In [87]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

99/99 [==============================] - 0s 81us/step
combination test accuracy: 77.78%


In [88]:
#### improve neural network on over-sampling data
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [89]:
model2_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [90]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 557us/step - loss: 17.1972 - accuracy: 0.5325 - val_loss: 0.7259 - val_accuracy: 0.5556
Epoch 2/1000
231/231 [==============================] - 0s 110us/step - loss: 0.6798 - accuracy: 0.5887 - val_loss: 0.6744 - val_accuracy: 0.6465
Epoch 3/1000
231/231 [==============================] - 0s 130us/step - loss: 0.6634 - accuracy: 0.6494 - val_loss: 0.7673 - val_accuracy: 0.5758
Epoch 4/1000
231/231 [==============================] - 0s 122us/step - loss: 0.6794 - accuracy: 0.6364 - val_loss: 0.6972 - val_accuracy: 0.5960
Epoch 5/1000
231/231 [==============================] - 0s 122us/step - loss: 0.6542 - accuracy: 0.6537 - val_loss: 0.6489 - val_accuracy: 0.6970
Epoch 6/1000
231/231 [==============================] - 0s 125us/step - loss: 0.6402 - accuracy: 0.7186 - val_loss: 0.6499 - val_accuracy: 0.7273
Epoch 7/1000
231/231 [==============================] - 0s 142us/step - loss: 

231/231 [==============================] - 0s 121us/step - loss: 0.5084 - accuracy: 0.7359 - val_loss: 0.5271 - val_accuracy: 0.7475
Epoch 57/1000
231/231 [==============================] - 0s 124us/step - loss: 0.4987 - accuracy: 0.7403 - val_loss: 0.5222 - val_accuracy: 0.7778
Epoch 58/1000
231/231 [==============================] - 0s 114us/step - loss: 0.4992 - accuracy: 0.7532 - val_loss: 0.5168 - val_accuracy: 0.7677
Epoch 59/1000
231/231 [==============================] - 0s 123us/step - loss: 0.5124 - accuracy: 0.7446 - val_loss: 0.5252 - val_accuracy: 0.7677
Epoch 60/1000
231/231 [==============================] - 0s 120us/step - loss: 0.5300 - accuracy: 0.7446 - val_loss: 0.5330 - val_accuracy: 0.7677
Epoch 61/1000
231/231 [==============================] - 0s 113us/step - loss: 0.4981 - accuracy: 0.7359 - val_loss: 0.5201 - val_accuracy: 0.7778
Epoch 62/1000
231/231 [==============================] - 0s 116us/step - loss: 0.5040 - accuracy: 0.7403 - val_loss: 0.5358 - val_ac

231/231 [==============================] - 0s 123us/step - loss: 0.4760 - accuracy: 0.7532 - val_loss: 0.5059 - val_accuracy: 0.7475
Epoch 112/1000
231/231 [==============================] - 0s 131us/step - loss: 0.4681 - accuracy: 0.7532 - val_loss: 0.5000 - val_accuracy: 0.7475
Epoch 113/1000
231/231 [==============================] - 0s 150us/step - loss: 0.4809 - accuracy: 0.7662 - val_loss: 0.5010 - val_accuracy: 0.7879
Epoch 114/1000
231/231 [==============================] - 0s 126us/step - loss: 0.4762 - accuracy: 0.7489 - val_loss: 0.5162 - val_accuracy: 0.7980
Epoch 115/1000
231/231 [==============================] - 0s 127us/step - loss: 0.4728 - accuracy: 0.7662 - val_loss: 0.5298 - val_accuracy: 0.7374
Epoch 116/1000
231/231 [==============================] - 0s 118us/step - loss: 0.4894 - accuracy: 0.7489 - val_loss: 0.5202 - val_accuracy: 0.7576
Epoch 117/1000
231/231 [==============================] - 0s 116us/step - loss: 0.4755 - accuracy: 0.7532 - val_loss: 0.5078 - 

231/231 [==============================] - 0s 130us/step - loss: 0.4614 - accuracy: 0.7662 - val_loss: 0.4907 - val_accuracy: 0.7879
Epoch 167/1000
231/231 [==============================] - 0s 121us/step - loss: 0.4422 - accuracy: 0.7835 - val_loss: 0.6002 - val_accuracy: 0.7475
Epoch 168/1000
231/231 [==============================] - 0s 126us/step - loss: 0.4830 - accuracy: 0.7532 - val_loss: 0.5168 - val_accuracy: 0.7879
Epoch 169/1000
231/231 [==============================] - 0s 118us/step - loss: 0.4515 - accuracy: 0.7619 - val_loss: 0.4895 - val_accuracy: 0.7576
Epoch 170/1000
231/231 [==============================] - 0s 118us/step - loss: 0.4553 - accuracy: 0.7749 - val_loss: 0.4874 - val_accuracy: 0.7475
Epoch 171/1000
231/231 [==============================] - 0s 123us/step - loss: 0.4517 - accuracy: 0.7619 - val_loss: 0.4975 - val_accuracy: 0.7879
Epoch 172/1000
231/231 [==============================] - 0s 118us/step - loss: 0.4510 - accuracy: 0.7619 - val_loss: 0.5133 - 

231/231 [==============================] - 0s 125us/step - loss: 0.4293 - accuracy: 0.7749 - val_loss: 0.4866 - val_accuracy: 0.7576
Epoch 222/1000
231/231 [==============================] - 0s 125us/step - loss: 0.4302 - accuracy: 0.7922 - val_loss: 0.4993 - val_accuracy: 0.7778
Epoch 223/1000
231/231 [==============================] - 0s 125us/step - loss: 0.4283 - accuracy: 0.7879 - val_loss: 0.5190 - val_accuracy: 0.7778
Epoch 224/1000
231/231 [==============================] - 0s 129us/step - loss: 0.4315 - accuracy: 0.7749 - val_loss: 0.4798 - val_accuracy: 0.7677
Epoch 225/1000
231/231 [==============================] - 0s 121us/step - loss: 0.4306 - accuracy: 0.7792 - val_loss: 0.4836 - val_accuracy: 0.7677
Epoch 226/1000
231/231 [==============================] - 0s 122us/step - loss: 0.4399 - accuracy: 0.8009 - val_loss: 0.4750 - val_accuracy: 0.7677
Epoch 227/1000
231/231 [==============================] - 0s 125us/step - loss: 0.4402 - accuracy: 0.7706 - val_loss: 0.4835 - 

...

231/231 [==============================] - 0s 118us/step - loss: 0.3576 - accuracy: 0.8052 - val_loss: 0.4855 - val_accuracy: 0.7778
Epoch 772/1000
231/231 [==============================] - 0s 121us/step - loss: 0.3560 - accuracy: 0.7835 - val_loss: 0.4735 - val_accuracy: 0.7980
Epoch 773/1000
231/231 [==============================] - 0s 146us/step - loss: 0.3503 - accuracy: 0.8052 - val_loss: 0.4676 - val_accuracy: 0.7677
Epoch 774/1000
231/231 [==============================] - 0s 300us/step - loss: 0.3527 - accuracy: 0.8052 - val_loss: 0.9774 - val_accuracy: 0.6061
Epoch 775/1000
231/231 [==============================] - 0s 177us/step - loss: 0.4215 - accuracy: 0.7662 - val_loss: 0.4708 - val_accuracy: 0.7778
Epoch 776/1000
231/231 [==============================] - 0s 156us/step - loss: 0.3606 - accuracy: 0.8052 - val_loss: 0.4728 - val_accuracy: 0.7677
Epoch 777/1000
231/231 [==============================] - 0s 176us/step - loss: 0.3584 - accuracy: 0.7879 - val_loss: 0.5015 - 

231/231 [==============================] - 0s 116us/step - loss: 0.3833 - accuracy: 0.7792 - val_loss: 0.4901 - val_accuracy: 0.7879
Epoch 827/1000
231/231 [==============================] - 0s 112us/step - loss: 0.3599 - accuracy: 0.8052 - val_loss: 0.4670 - val_accuracy: 0.7879
Epoch 828/1000
231/231 [==============================] - 0s 129us/step - loss: 0.3520 - accuracy: 0.8182 - val_loss: 0.4676 - val_accuracy: 0.7677
Epoch 829/1000
231/231 [==============================] - 0s 122us/step - loss: 0.3475 - accuracy: 0.8009 - val_loss: 0.5090 - val_accuracy: 0.8182
Epoch 830/1000
231/231 [==============================] - 0s 121us/step - loss: 0.3585 - accuracy: 0.8095 - val_loss: 0.4624 - val_accuracy: 0.7778
Epoch 831/1000
231/231 [==============================] - 0s 120us/step - loss: 0.3416 - accuracy: 0.8268 - val_loss: 0.5002 - val_accuracy: 0.7778
Epoch 832/1000
231/231 [==============================] - 0s 118us/step - loss: 0.3653 - accuracy: 0.8009 - val_loss: 0.4883 - 

231/231 [==============================] - 0s 119us/step - loss: 0.3531 - accuracy: 0.8052 - val_loss: 0.4738 - val_accuracy: 0.7778
Epoch 882/1000
231/231 [==============================] - 0s 121us/step - loss: 0.3801 - accuracy: 0.7879 - val_loss: 0.4936 - val_accuracy: 0.7879
Epoch 883/1000
231/231 [==============================] - 0s 125us/step - loss: 0.3639 - accuracy: 0.8095 - val_loss: 0.4740 - val_accuracy: 0.7778
Epoch 884/1000
231/231 [==============================] - 0s 119us/step - loss: 0.3714 - accuracy: 0.7922 - val_loss: 0.4710 - val_accuracy: 0.7879
Epoch 885/1000
231/231 [==============================] - 0s 126us/step - loss: 0.3562 - accuracy: 0.7965 - val_loss: 0.4820 - val_accuracy: 0.7879
Epoch 886/1000
231/231 [==============================] - 0s 119us/step - loss: 0.3528 - accuracy: 0.7965 - val_loss: 0.4670 - val_accuracy: 0.7879
Epoch 887/1000
231/231 [==============================] - 0s 129us/step - loss: 0.3992 - accuracy: 0.7662 - val_loss: 0.4675 - 

231/231 [==============================] - 0s 116us/step - loss: 0.3490 - accuracy: 0.8052 - val_loss: 0.4703 - val_accuracy: 0.7879
Epoch 937/1000
231/231 [==============================] - 0s 120us/step - loss: 0.3527 - accuracy: 0.8052 - val_loss: 0.5099 - val_accuracy: 0.7980
Epoch 938/1000
231/231 [==============================] - 0s 123us/step - loss: 0.3541 - accuracy: 0.7965 - val_loss: 0.4647 - val_accuracy: 0.7778
Epoch 939/1000
231/231 [==============================] - 0s 128us/step - loss: 0.3498 - accuracy: 0.8052 - val_loss: 0.4862 - val_accuracy: 0.7980
Epoch 940/1000
231/231 [==============================] - 0s 123us/step - loss: 0.3612 - accuracy: 0.8009 - val_loss: 0.4649 - val_accuracy: 0.7879
Epoch 941/1000
231/231 [==============================] - 0s 116us/step - loss: 0.3444 - accuracy: 0.8095 - val_loss: 0.4789 - val_accuracy: 0.7879
Epoch 942/1000
231/231 [==============================] - 0s 122us/step - loss: 0.3571 - accuracy: 0.8052 - val_loss: 0.5229 - 

231/231 [==============================] - 0s 122us/step - loss: 0.3713 - accuracy: 0.8052 - val_loss: 0.4914 - val_accuracy: 0.7677
Epoch 992/1000
231/231 [==============================] - 0s 125us/step - loss: 0.3726 - accuracy: 0.7965 - val_loss: 0.4881 - val_accuracy: 0.7576
Epoch 993/1000
231/231 [==============================] - 0s 117us/step - loss: 0.3618 - accuracy: 0.8052 - val_loss: 0.4708 - val_accuracy: 0.7778
Epoch 994/1000
231/231 [==============================] - 0s 117us/step - loss: 0.3786 - accuracy: 0.8052 - val_loss: 0.4958 - val_accuracy: 0.7778
Epoch 995/1000
231/231 [==============================] - 0s 125us/step - loss: 0.3699 - accuracy: 0.8009 - val_loss: 0.4848 - val_accuracy: 0.7677
Epoch 996/1000
231/231 [==============================] - 0s 127us/step - loss: 0.3663 - accuracy: 0.8009 - val_loss: 0.4822 - val_accuracy: 0.7576
Epoch 997/1000
231/231 [==============================] - 0s 120us/step - loss: 0.3687 - accuracy: 0.7965 - val_loss: 0.4775 - 

In [91]:
acc_test_over3 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over3*100))

99/99 [==============================] - 0s 78us/step
combination test accuracy: 76.77%


In [92]:
############ Logistic Regression ############

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [94]:
###### logistics
log = LogisticRegression(random_state=123)
log.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
y_pred = log.predict(X_test)
print('logistic test accuracy %.2f%%' % (log.score(X_test, y_test)*100))

logistic test accuracy 76.62%


In [96]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [97]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 94.92%


In [98]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [99]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 85.86%


In [100]:
############## Random Forest ##############

In [101]:
###### random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=123)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [102]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)
print('test accuracy: %.2f%%' % (accuracy_score(y_test, y_pred)*100))

test accuracy: 76.62%


In [103]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [104]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [105]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [106]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 86.87%


In [107]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [108]:
## random forest model with CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv = RandomForestClassifier(n_estimators=100, random_state=123)

accs = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=5)
print(accs)
print(accs.mean())

[0.80555556 0.86111111 0.83333333 0.85714286 0.77142857]
0.8257142857142858


In [109]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.96428571 0.96428571 0.85714286 0.92592593 0.88461538]
0.9192511192511192


In [110]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.89361702 0.84782609 0.82608696 0.73913043 0.84782609]
0.8308973172987975
